In [1]:
import pandas as pd
import bs4 as bs # pulling data out of HTML and XML files.
import urllib.request # opening and reading URLs
import re # pattern matching

In [14]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/Users/jieluyao/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Load the master file

In [2]:
h = pd.read_csv("/Users/jieluyao/Documents/GitHub/datasets/candidates/2022/wmpcand_101422_wmpid.csv")

In [3]:
h.columns

Index(['wmpid', 'genelect_cd', 'CurrCand', 'cand_name', 'cand_id',
       'cand_office', 'cand_office_st', 'cand_office_dist',
       'cand_party_affiliation', 'cand_incumbent_challenger_open_s',
       'dateadded_cd', 'office_wapo', 'result_wapo', 'primarydate_wapo',
       'trumpe_wapo', 'gender_wmp', 'gender_crp', 'latino_wmp', 'latino_crp',
       'race_wmp', 'race_crp1', 'race_crp2', 'race_crpmena', 'full_name',
       'first_name', 'last_name'],
      dtype='object')

In [4]:
h.genelect_cd.value_counts(dropna=False)

0    2828
1    1468
Name: genelect_cd, dtype: int64

In [5]:
h = h.loc[(h['genelect_cd']==1)&(h['cand_office']=='H')]

In [6]:
h.shape

(1296, 26)

In [8]:
h = h[['wmpid', 'cand_name', 'full_name', 'cand_office_st', 'cand_office_dist', 'cand_party_affiliation']]

In [9]:
h.shape

(1296, 6)

In [10]:
len(h.full_name.unique())

1296

In [11]:
h.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
0,WMPID21,"CARL, JERRY LEE, JR",Jerry Carl,AL,1.0,REP
1,WMPID24,"HARVEY-HALL, PHYLLIS",Phyllis Harvey-Hall,AL,2.0,DEM
2,WMPID27,"SEWELL, TERRI A.",Terri A Sewell,AL,7.0,DEM
3,WMPID28,"CRAWFORD, ERIC ALAN RICK",Rick Crawford,AR,1.0,REP
4,WMPID30,"WOMACK, STEVE",Steve Womack,AR,3.0,REP


In [12]:
h.cand_office_dist.unique()

array([ 1.,  2.,  7.,  3.,  4.,  5.,  6., 23., 10.,  8., 14., 17., 18.,
       19., 27., 30., 28., 35., 38., 36., 46., 48., 45.,  0., 11.,  9.,
       13., 24., 25., 12., 15., 16., 20., 22., 26., 34., 37., 21., 29.,
       31., 32., 33., 41., 40., 39., 42., 47., 52., 50., 51., 43., 44.,
       49.])

In [13]:
h.cand_office_st.value_counts(dropna=False)

FL    165
NY    113
CA    105
TX     92
NJ     48
MI     47
IL     40
GA     36
PA     35
OH     33
TN     33
CO     32
NC     30
VA     28
MA     28
IN     24
MO     23
AZ     23
MD     22
WA     22
LA     21
WI     21
OR     18
MN     18
SC     18
AL     18
OK     14
NH     14
KY     14
NV     13
UT     13
CT     12
MS     12
AR     12
IA     11
MT      8
KS      8
NE      8
NM      7
ME      6
HI      5
RI      5
WV      5
ID      5
DE      5
AK      4
WY      4
DC      4
ND      3
GU      3
VT      3
SD      2
VI      1
AS      1
MP      1
Name: cand_office_st, dtype: int64

In [47]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [1495]:
len(res1.columns)

12

In [1496]:
len(res2.columns)

12

In [1509]:
res = pd.concat([res1, res2])

In [1510]:
res.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID2014,"PALIN, SARAH",Sarah Palin,AK,0.0,REP,Sarah Palin,Sarah Palin,https://s3.amazonaws.com/ballotpedia-api4/file...,Sarah Palin,Sarah_Palin.PNG,both
1,WMPID4003,"BEGICH, NICHOLAS III",Nicholas III Begich,AK,0.0,REP,Nicholas Begich,Nicholas Begich,https://s3.amazonaws.com/ballotpedia-api4/file...,Nicholas Begich,Nick-Begich.PNG,both
2,WMPID4308,"PELTOLA, MARY",Mary Peltola,AK,0.0,DEM,Mary Peltola,Mary Peltola,https://s3.amazonaws.com/ballotpedia-api4/file...,Mary Peltola,Mary-Peltola.PNG,both
3,WMPID4360,"SWEENEY, TARA M",Tara Sweeney,AK,0.0,REP,,NaN,NaN,NaN,NaN,left_only
0,WMPID5148,"ROGERS, DAVID L.",David Rogers,DE,0.0,OTH,David Rogers,David Rogers (Nonpartisan Party) \n\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,David Rogers,DavidRogers.jpg,both


In [1511]:
res._merge.value_counts(dropna=False)

both          903
left_only     383
right_only      0
Name: _merge, dtype: int64

In [1512]:
res.to_csv("bp2022_house_scraped_face.csv", index=False)

In [1500]:
del res['_merge']

In [1504]:
master = h[['wmpid', 'cand_office_st']]

In [1505]:
dat = master.merge(res, on='wmpid', how='left', indicator=True)

In [1506]:
dat._merge.value_counts()

both          1286
left_only       10
right_only       0
Name: _merge, dtype: int64

In [1507]:
dat.loc[dat['_merge']=='left_only']

,wmpid,cand_office_st_x,cand_name,full_name,cand_office_st_y,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
31,WMPID115,DC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
309,WMPID2101,DC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
310,WMPID4991,DC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
447,WMPID3894,GU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
448,WMPID4351,GU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
953,WMPID858,VI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
987,WMPID865,AS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1163,WMPID1083,DC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1176,WMPID1100,GU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1216,WMPID1163,MP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


### House

In [3]:
#Download the website's html
source = urllib.request.urlopen("https://ballotpedia.org/United_States_House_of_Representatives_elections,_2022").read()

In [4]:
#Parse the html
#This makes it look nice and structured for us, and much easier to read for the program
soup = bs.BeautifulSoup(source)

In [6]:
urls_css = "center small a"

In [7]:
urls = soup.select(urls_css)

In [11]:
extracted_urls = [url["href"] for url in urls]
len(extracted_urls)

50

In [9]:
full_urls = ["https://ballotpedia.org" + url_suffix for url_suffix in extracted_urls]

In [10]:
full_urls

['https://ballotpedia.org/United_States_House_of_Representatives_elections_in_Alabama,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_election_in_Alaska,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_elections_in_Arizona,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_elections_in_Arkansas,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_elections_in_California,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_elections_in_Colorado,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_elections_in_Connecticut,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_election_in_Delaware,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_elections_in_Florida,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_elections_in_Georgia,_2022',
 'https://ballotpedia.org/United_States_House_of_Repr

#### Distinguish two types of structure 

In [13]:
urls_1 = []
urls_2 = []

for i in full_urls:
  source = urllib.request.urlopen(i).read()
  soup = bs.BeautifulSoup(source)
  res_css = "#District_1"
  res = soup.select(res_css)
  if len(res) == 0:
    urls_1.append(i)
  else:
    urls_2.append(i)

In [14]:
urls_1

['https://ballotpedia.org/United_States_House_of_Representatives_election_in_Alaska,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_election_in_Delaware,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_election_in_North_Dakota,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_election_in_South_Dakota,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_election_in_Vermont,_2022',
 'https://ballotpedia.org/United_States_House_of_Representatives_election_in_Wyoming,_2022']

In [15]:
len(urls_2)

44

#### States with one layer

In [187]:
d1 = pd.DataFrame(columns=['bp_name_raw', 'bp_url'])

In [188]:
for i in urls_1[5:6]:
  print(i)
  images = []
  bp_names = []
  source = urllib.request.urlopen(i).read()
  soup = bs.BeautifulSoup(source)
  img_css = ".image-candidate-thumbnail"
  img = soup.select(img_css)
  img = img[0:5]
  images.extend(list([i['src'] for i in img]))
  n_css = "td.votebox-results-cell--text"
  nm = soup.select(n_css)
  nm = nm[0:5]
  bp_names.extend(list([url.text for url in nm]))
  dic = {'bp_name_raw':bp_names,'bp_url':images}
  df = pd.DataFrame(dic)
  d1=d1.append(df)

https://ballotpedia.org/United_States_House_of_Representatives_election_in_Wyoming,_2022


In [189]:
d1

,bp_name_raw,bp_url
0,Harriet Hageman (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...
1,Lynnette Grey Bull (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...
2,Richard Brubaker (L) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...
3,Marissa Selvig (Constitution Party) \n,https://s3.amazonaws.com/ballotpedia-api4/file...
4,Other/Write-in votes,https://s3.amazonaws.com/ballotpedia-api4/file...


In [190]:
df2 = d1['bp_name_raw'].str.rsplit("(", n=1, expand=True)

In [191]:
df2 = df2.rename(columns={0:'name'})

In [192]:
df3 =df2['name'].str.rsplit("(", n=1, expand=True)
df3 = df3.rename(columns={0:'name'})

In [193]:
df3['bp_name'] = df3['name'].str.replace("\n", "").replace("\t", "")

In [194]:
df3['bp_name']

0        Harriet Hageman 
1     Lynnette Grey Bull 
2       Richard Brubaker 
3         Marissa Selvig 
4    Other/Write-in votes
Name: bp_name, dtype: object

In [195]:
df['bp_name']=df3['bp_name']

In [196]:
df['bp_url_filename'] = df['bp_url'].str.replace("https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/100/100/", '')

<ipython-input-196-69e59d51501f>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['bp_url_filename'] = df['bp_url'].str.replace("https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/100/100/", '')


In [197]:
df['bp_name'] = df['bp_name'].str.strip()

In [198]:
df = df.drop_duplicates(subset=['bp_name'], keep='first')

In [199]:
df

,bp_name_raw,bp_url,bp_name,bp_url_filename
0,Harriet Hageman (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Harriet Hageman,Harriet-Hageman.PNG
1,Lynnette Grey Bull (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Lynnette Grey Bull,LynnetteGreyBullWY.png
2,Richard Brubaker (L) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Richard Brubaker,Richard_Brubaker.jpg
3,Marissa Selvig (Constitution Party) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Marissa Selvig,Marissa_Selvig.PNG
4,Other/Write-in votes,https://s3.amazonaws.com/ballotpedia-api4/file...,Other/Write-in votes,LynnetteGreyBullWY.png


In [200]:
WY = h.loc[h['cand_office_st']=='WY']

In [201]:
WY

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
369,WMPID767,"GREYBULL, LYNNETTE",Lynnette Grey Bull,WY,0.0,DEM
3156,WMPID5149,"BRUBAKER, RICHARD P.",Richard Brubaker,WY,0.0,LIB
3158,WMPID4061,"HAGEMAN, HARRIET",Harriet Hageman,WY,0.0,REP
3160,WMPID4514,"SELVIG, MARISSA JOY",Marissa Joy Selvig,WY,0.0,CON


In [202]:
wy = fuzzy_merge(WY, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [203]:
wy.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
369,WMPID767,"GREYBULL, LYNNETTE",Lynnette Grey Bull,WY,0.0,DEM,Lynnette Grey Bull
3156,WMPID5149,"BRUBAKER, RICHARD P.",Richard Brubaker,WY,0.0,LIB,Richard Brubaker
3158,WMPID4061,"HAGEMAN, HARRIET",Harriet Hageman,WY,0.0,REP,Harriet Hageman
3160,WMPID4514,"SELVIG, MARISSA JOY",Marissa Joy Selvig,WY,0.0,CON,Marissa Selvig


In [205]:
wy= wy.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [206]:
wy.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID767,"GREYBULL, LYNNETTE",Lynnette Grey Bull,WY,0.0,DEM,Lynnette Grey Bull,Lynnette Grey Bull (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Lynnette Grey Bull,LynnetteGreyBullWY.png,both
1,WMPID5149,"BRUBAKER, RICHARD P.",Richard Brubaker,WY,0.0,LIB,Richard Brubaker,Richard Brubaker (L) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Richard Brubaker,Richard_Brubaker.jpg,both
2,WMPID4061,"HAGEMAN, HARRIET",Harriet Hageman,WY,0.0,REP,Harriet Hageman,Harriet Hageman (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Harriet Hageman,Harriet-Hageman.PNG,both
3,WMPID4514,"SELVIG, MARISSA JOY",Marissa Joy Selvig,WY,0.0,CON,Marissa Selvig,Marissa Selvig (Constitution Party) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Marissa Selvig,Marissa_Selvig.PNG,both


In [180]:
VT = h.loc[h['cand_office_st']=='VT']

In [181]:
VT

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
3050,WMPID5069,"NOLFI, BARBARA",Barbara Nolfi,VT,0.0,PRO
3052,WMPID2308,"BALINT, REBECCA BECCA",Rebecca Becca Balint,VT,0.0,DEM
3055,WMPID2620,"MADDEN, LIAM",Liam Madden,VT,0.0,IND


In [182]:
vt = fuzzy_merge(VT, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [184]:
vt.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
3050,WMPID5069,"NOLFI, BARBARA",Barbara Nolfi,VT,0.0,PRO,
3052,WMPID2308,"BALINT, REBECCA BECCA",Rebecca Becca Balint,VT,0.0,DEM,Becca Balint
3055,WMPID2620,"MADDEN, LIAM",Liam Madden,VT,0.0,IND,Liam Madden


In [185]:
vt= vt.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [186]:
vt.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID5069,"NOLFI, BARBARA",Barbara Nolfi,VT,0.0,PRO,,NaN,NaN,NaN,NaN,left_only
1,WMPID2308,"BALINT, REBECCA BECCA",Rebecca Becca Balint,VT,0.0,DEM,Becca Balint,Becca Balint (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Becca Balint,Becca-Balint.PNG,both
2,WMPID2620,"MADDEN, LIAM",Liam Madden,VT,0.0,IND,Liam Madden,Liam Madden (R) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Liam Madden,Liam_Madden.png,both


In [161]:
SD = h.loc[h['cand_office_st']=='SD']

In [162]:
SD

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
2632,WMPID5122,"DUPREL, COLLIN",Collin Duprel,SD,0.0,LIB
3560,WMPID1231,"JOHNSON, DUSTY",Dusty Johnson,SD,0.0,REP


In [163]:
sd = fuzzy_merge(SD, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [164]:
sd.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
2632,WMPID5122,"DUPREL, COLLIN",Collin Duprel,SD,0.0,LIB,Collin Duprel
3560,WMPID1231,"JOHNSON, DUSTY",Dusty Johnson,SD,0.0,REP,Dusty Johnson


In [165]:
sd = sd.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [166]:
sd.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID5122,"DUPREL, COLLIN",Collin Duprel,SD,0.0,LIB,Collin Duprel,Collin Duprel (L) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Collin Duprel,Collin_Duprel2.jpg,both
1,WMPID1231,"JOHNSON, DUSTY",Dusty Johnson,SD,0.0,REP,Dusty Johnson,Dusty Johnson (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Dusty Johnson,Dusty_Johnson_resize.jpg,both


In [141]:
ND = h.loc[h['cand_office_st']=='ND']

In [142]:
ND.shape

(3, 6)

In [143]:
ND

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
1969,WMPID2318,"HAUGEN, MARK",Mark Haugen,ND,0.0,DEM
1970,WMPID5103,"MUND, CARA",Cara Mund,ND,0.0,(I)
3502,WMPID1171,"ARMSTRONG, KELLY",Kelly Armstrong,ND,0.0,REP


In [144]:
nd = fuzzy_merge(ND, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [145]:
nd.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
1969,WMPID2318,"HAUGEN, MARK",Mark Haugen,ND,0.0,DEM,Mark Haugen
1970,WMPID5103,"MUND, CARA",Cara Mund,ND,0.0,(I),Cara Mund
3502,WMPID1171,"ARMSTRONG, KELLY",Kelly Armstrong,ND,0.0,REP,Kelly Armstrong


In [146]:
nd = nd.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [147]:
nd.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID2318,"HAUGEN, MARK",Mark Haugen,ND,0.0,DEM,Mark Haugen,Mark Haugen \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Mark Haugen,KELLY_ARMSTRONG.jpg,both
1,WMPID5103,"MUND, CARA",Cara Mund,ND,0.0,(I),Cara Mund,Cara Mund (Independent) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Cara Mund,Cara_Mund.PNG,both
2,WMPID1171,"ARMSTRONG, KELLY",Kelly Armstrong,ND,0.0,REP,Kelly Armstrong,Kelly Armstrong (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Kelly Armstrong,KELLY_ARMSTRONG.jpg,both


In [122]:
DE = h.loc[h['cand_office_st']=='DE']

In [123]:
DE.shape

(5, 6)

In [124]:
DE

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
65,WMPID5148,"ROGERS, DAVID L.",David Rogers,DE,0.0,OTH
910,WMPID5144,"MCNUTT, CODY",Cody Mcnutt,DE,0.0,LIB
3276,WMPID959,"BLUNT ROCHESTER, LISA",Lisa Blunt Rochester,DE,0.0,DEM
3277,WMPID5139,"WALKER, SCOTT",Scott Walker,DE,0.0,W
3424,WMPID1084,"MURPHY, LEE",Lee Murphy,DE,0.0,REP


In [125]:
de = fuzzy_merge(DE, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [126]:
de = de.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [127]:
de.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID5148,"ROGERS, DAVID L.",David Rogers,DE,0.0,OTH,David Rogers,David Rogers (Nonpartisan Party) \n\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,David Rogers,DavidRogers.jpg,both
1,WMPID5144,"MCNUTT, CODY",Cody Mcnutt,DE,0.0,LIB,Cody McNutt,Cody McNutt (L) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Cody McNutt,Cody-McNutt.PNG,both
2,WMPID959,"BLUNT ROCHESTER, LISA",Lisa Blunt Rochester,DE,0.0,DEM,Lisa Blunt Rochester,Lisa Blunt Rochester (D) \n\t\t\t\t\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,Lisa Blunt Rochester,Lisa_Blunt_Rochester.jpg,both
3,WMPID5139,"WALKER, SCOTT",Scott Walker,DE,0.0,W,Scott Walker,Scott Walker (Independent) (Write-in) \n\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Scott Walker,Scott_WalkerDE.jpeg,both
4,WMPID1084,"MURPHY, LEE",Lee Murphy,DE,0.0,REP,Lee Murphy,Lee Murphy (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Lee Murphy,Lee_Murphy_for_U.S._Congress_Delaware_photo.JPG,both


In [44]:
AK = h.loc[h['cand_office_st']=='AK']

In [45]:
AK.shape

(4, 6)

In [46]:
AK

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
375,WMPID2014,"PALIN, SARAH",Sarah Palin,AK,0.0,REP
411,WMPID4003,"BEGICH, NICHOLAS III",Nicholas III Begich,AK,0.0,REP
417,WMPID4308,"PELTOLA, MARY",Mary Peltola,AK,0.0,DEM
419,WMPID4360,"SWEENEY, TARA M",Tara Sweeney,AK,0.0,REP


In [103]:
ak = fuzzy_merge(AK, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [104]:
ak

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
375,WMPID2014,"PALIN, SARAH",Sarah Palin,AK,0.0,REP,Sarah Palin
411,WMPID4003,"BEGICH, NICHOLAS III",Nicholas III Begich,AK,0.0,REP,Nicholas Begich
417,WMPID4308,"PELTOLA, MARY",Mary Peltola,AK,0.0,DEM,Mary Peltola
419,WMPID4360,"SWEENEY, TARA M",Tara Sweeney,AK,0.0,REP,


In [105]:
ak = ak.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [106]:
ak.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID2014,"PALIN, SARAH",Sarah Palin,AK,0.0,REP,Sarah Palin,Sarah Palin,https://s3.amazonaws.com/ballotpedia-api4/file...,Sarah Palin,Sarah_Palin.PNG,both
1,WMPID4003,"BEGICH, NICHOLAS III",Nicholas III Begich,AK,0.0,REP,Nicholas Begich,Nicholas Begich,https://s3.amazonaws.com/ballotpedia-api4/file...,Nicholas Begich,Nick-Begich.PNG,both
2,WMPID4308,"PELTOLA, MARY",Mary Peltola,AK,0.0,DEM,Mary Peltola,Mary Peltola,https://s3.amazonaws.com/ballotpedia-api4/file...,Mary Peltola,Mary-Peltola.PNG,both
3,WMPID4360,"SWEENEY, TARA M",Tara Sweeney,AK,0.0,REP,,NaN,NaN,NaN,NaN,left_only


#### Combine six states' results


In [208]:
res1 = pd.concat([ak, de, nd, sd, vt, wy])

In [209]:
res1.shape

(21, 12)

In [211]:
res1

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID2014,"PALIN, SARAH",Sarah Palin,AK,0.0,REP,Sarah Palin,Sarah Palin,https://s3.amazonaws.com/ballotpedia-api4/file...,Sarah Palin,Sarah_Palin.PNG,both
1,WMPID4003,"BEGICH, NICHOLAS III",Nicholas III Begich,AK,0.0,REP,Nicholas Begich,Nicholas Begich,https://s3.amazonaws.com/ballotpedia-api4/file...,Nicholas Begich,Nick-Begich.PNG,both
2,WMPID4308,"PELTOLA, MARY",Mary Peltola,AK,0.0,DEM,Mary Peltola,Mary Peltola,https://s3.amazonaws.com/ballotpedia-api4/file...,Mary Peltola,Mary-Peltola.PNG,both
3,WMPID4360,"SWEENEY, TARA M",Tara Sweeney,AK,0.0,REP,,NaN,NaN,NaN,NaN,left_only
0,WMPID5148,"ROGERS, DAVID L.",David Rogers,DE,0.0,OTH,David Rogers,David Rogers (Nonpartisan Party) \n\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,David Rogers,DavidRogers.jpg,both
1,WMPID5144,"MCNUTT, CODY",Cody Mcnutt,DE,0.0,LIB,Cody McNutt,Cody McNutt (L) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Cody McNutt,Cody-McNutt.PNG,both
2,WMPID959,"BLUNT ROCHESTER, LISA",Lisa Blunt Rochester,DE,0.0,DEM,Lisa Blunt Rochester,Lisa Blunt Rochester (D) \n\t\t\t\t\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,Lisa Blunt Rochester,Lisa_Blunt_Rochester.jpg,both
3,WMPID5139,"WALKER, SCOTT",Scott Walker,DE,0.0,W,Scott Walker,Scott Walker (Independent) (Write-in) \n\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Scott Walker,Scott_WalkerDE.jpeg,both
4,WMPID1084,"MURPHY, LEE",Lee Murphy,DE,0.0,REP,Lee Murphy,Lee Murphy (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Lee Murphy,Lee_Murphy_for_U.S._Congress_Delaware_photo.JPG,both
0,WMPID2318,"HAUGEN, MARK",Mark Haugen,ND,0.0,DEM,Mark Haugen,Mark Haugen \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Mark Haugen,KELLY_ARMSTRONG.jpg,both


#### States with two layers

In [1460]:
images = []
bp_name_raw = []

In [1461]:
for i in urls_2[28:29]:
  print(i)
  source = urllib.request.urlopen(i).read()
  soup = bs.BeautifulSoup(source)
  dist_css = "dd dl dd i a"
  dist = soup.select(dist_css)
  # urls_css = "center small a"
  # urls = soup.select(urls_css)
  extracted_urls = [i["href"] for i in dist]
  st_urls = ["https://ballotpedia.org" + url_suffix for url_suffix in extracted_urls]
  print(st_urls)
  for i in st_urls:
    #if '27s' in i:
    if '27' in i:
        source2 = urllib.request.urlopen(i).read()
        soup2 = bs.BeautifulSoup(source2)
        img_css = ".image-candidate-thumbnail"
        img = soup2.select(img_css)
        img = img[0:5]
        #print(img)
        images.extend(list([i['src'] for i in img]))
        n_css = "td.votebox-results-cell--text"
        nm = soup2.select(n_css)
        nm = nm[0:5]
        #print(nm)
        bp_name_raw.extend(list([url.text for url in nm]))

https://ballotpedia.org/United_States_House_of_Representatives_elections_in_New_Mexico,_2022
['https://ballotpedia.org/New_Mexico%27s_1st_Congressional_District_election,_2022', 'https://ballotpedia.org/New_Mexico%27s_2nd_Congressional_District_election,_2022', 'https://ballotpedia.org/New_Mexico%27s_3rd_Congressional_District_election,_2022', 'https://ballotpedia.org/Race_rating_definitions_and_methods', 'https://ballotpedia.org/Primary_election_competitiveness_in_state_and_federal_government,_2022', 'https://ballotpedia.org/Presidential_election_in_New_Mexico,_2020', 'https://ballotpedia.org/The_Cook_Political_Report%27s_Partisan_Voter_Index']


In [1462]:
len(images)

15

In [1463]:
len(bp_name_raw)

15

In [1464]:
h_dic = {'bp_name_raw':bp_name_raw,'bp_url':images}

In [1465]:
df = pd.DataFrame(h_dic)

In [1466]:
df2 = df['bp_name_raw'].str.rsplit("(", n=1, expand=True)

In [1467]:
df2 = df2.rename(columns={0:'name'})

In [1468]:
df3 =df2['name'].str.rsplit("(", n=1, expand=True)
df3 = df3.rename(columns={0:'name'})

In [1469]:
df3['bp_name'] = df3['name'].str.replace("\n", "").replace("\t", "")

In [1470]:
df3['bp_name']

0       Melanie Ann Stansbury 
1      Michelle Garcia Holmes 
2           Victoria Gonzales 
3        Melanie Ann Stansbury
4       Michelle Garcia Holmes
5             Gabriel Vasquez 
6              Yvette Herrell 
7                 Eliseo Luna 
8              Gabriel Vasquez
9               Darshan Patel 
10     Teresa Leger Fernandez 
11    Alexis Martinez Johnson 
12      Teresa Leger Fernandez
13     Alexis Martinez Johnson
14     Teresa Leger Fernandez 
Name: bp_name, dtype: object

In [1471]:
df['bp_name']=df3['bp_name']

In [1472]:
df['bp_url_filename'] = df['bp_url'].str.replace("https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/100/100/", '')

<ipython-input-1472-69e59d51501f>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['bp_url_filename'] = df['bp_url'].str.replace("https://s3.amazonaws.com/ballotpedia-api4/files/thumbs/100/100/", '')


In [1473]:
df['bp_name'] = df['bp_name'].str.strip()

In [1474]:
df = df.drop_duplicates(subset=['bp_name'], keep='first')

In [1475]:
df

,bp_name_raw,bp_url,bp_name,bp_url_filename
0,Melanie Ann Stansbury (D) \n\t\t\t\t\t\t\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Melanie Ann Stansbury,_Melanie-Stansbury_.jpg
1,Michelle Garcia Holmes (R) \n\t\t\t\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,Michelle Garcia Holmes,w80182230_headshot.jpg
2,Victoria Gonzales (Independent) \n\t\t\t\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Victoria Gonzales,_Melanie-Stansbury_.jpg
5,Gabriel Vasquez (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Gabriel Vasquez,Vasquezgabe.jpg
6,Yvette Herrell (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Yvette Herrell,YvetteHerrell.jpg
7,Eliseo Luna (D) (Write-in) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Eliseo Luna,EliseoLuna2.jpg
9,Darshan Patel \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Darshan Patel,Darshan_Patel_2.jpg
10,Teresa Leger Fernandez (D) \n\t\t\t\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,Teresa Leger Fernandez,Teresa-Leger-Fernandez.PNG
11,Alexis Martinez Johnson (R) \n\t\t\t\t\t\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Alexis Martinez Johnson,80182230_alexis_professional.jpg


In [1487]:
res2 = pd.concat([al, az, ar, ca, co, ct, fl, ga, hi, xid, il, xin, ia, ks, ky,
                 la, me, md, ma, mi, mn, ms, mo, mt, ne, nv, nh, nj, nm, ny, nc, oh, ok,
                 xor, pa, ri, sc, tn, tx, ut, va, wa, wv, wi
                 ])

In [1488]:
res2.shape

(1265, 12)

In [1489]:
res2.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID21,"CARL, JERRY LEE, JR",Jerry Carl,AL,1.0,REP,Jerry Carl,Jerry Carl (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Jerry Carl,Jerry-Carl.PNG,both
1,WMPID24,"HARVEY-HALL, PHYLLIS",Phyllis Harvey-Hall,AL,2.0,DEM,Phyllis Harvey-Hall,Phyllis Harvey-Hall (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Phyllis Harvey-Hall,phyllis.jpg,both
2,WMPID27,"SEWELL, TERRI A.",Terri A Sewell,AL,7.0,DEM,Terri Sewell,Terri Sewell (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Terri Sewell,Terri_Sewell.jpg,both
3,WMPID4534,"SMITH, JONATHAN",Jonathan Smith,AL,2.0,LIB,,NaN,NaN,NaN,NaN,left_only
4,WMPID769,"ROGERS, MICHAEL",Mike D Rogers,AL,3.0,REP,Mike Rogers,Mike Rogers (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mike Rogers,Mike-Rogers.jpg,both


In [1491]:
res2.shape

(1265, 12)

In [1401]:
WI = h.loc[h['cand_office_st']=='WI']

In [1402]:
WI.shape

(21, 6)

In [1403]:
WI

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
362,WMPID753,"VAN ORDEN, DERRICK F. MR.",Derrick Van Orden,WI,3.0,REP
363,WMPID755,"FITZGERALD, SCOTT",Scott Fitzgerald,WI,5.0,REP
365,WMPID758,"TIFFANY, TOM",Tom Tiffany,WI,7.0,REP
3117,WMPID2029,"ROE, ANN",Ann Roe,WI,1.0,DEM
3119,WMPID861,"POCAN, MARK",Mark Pocan,WI,2.0,DEM
3120,WMPID3729,"BARRY, CHARITY",Charity Barry,WI,2.0,REP
3121,WMPID4549,"OLSEN, ERIK",Erik Olsen,WI,2.0,REP
3124,WMPID2537,"PFAFF, BRAD",Brad Pfaff,WI,3.0,DEM
3132,WMPID2197,"VAN SOMEREN, MIKE",Mike Van Someren,WI,5.0,DEM
3133,WMPID3315,"KIDD, JIM KIDD",Jim Kidd Kidd,WI,5.0,NNE


In [1406]:
wi = fuzzy_merge(WI, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1407]:
wi

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
362,WMPID753,"VAN ORDEN, DERRICK F. MR.",Derrick Van Orden,WI,3.0,REP,Derrick Van Orden
363,WMPID755,"FITZGERALD, SCOTT",Scott Fitzgerald,WI,5.0,REP,Scott Fitzgerald
365,WMPID758,"TIFFANY, TOM",Tom Tiffany,WI,7.0,REP,Tom Tiffany
3117,WMPID2029,"ROE, ANN",Ann Roe,WI,1.0,DEM,Ann Roe
3119,WMPID861,"POCAN, MARK",Mark Pocan,WI,2.0,DEM,Mark Pocan
3120,WMPID3729,"BARRY, CHARITY",Charity Barry,WI,2.0,REP,
3121,WMPID4549,"OLSEN, ERIK",Erik Olsen,WI,2.0,REP,Erik Olsen
3124,WMPID2537,"PFAFF, BRAD",Brad Pfaff,WI,3.0,DEM,Brad Pfaff
3132,WMPID2197,"VAN SOMEREN, MIKE",Mike Van Someren,WI,5.0,DEM,Mike Van Someren
3133,WMPID3315,"KIDD, JIM KIDD",Jim Kidd Kidd,WI,5.0,NNE,


In [1408]:
wi = wi.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1409]:
wi.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID753,"VAN ORDEN, DERRICK F. MR.",Derrick Van Orden,WI,3.0,REP,Derrick Van Orden,Derrick Van Orden (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Derrick Van Orden,Derrick-VanOrden.PNG,both
1,WMPID755,"FITZGERALD, SCOTT",Scott Fitzgerald,WI,5.0,REP,Scott Fitzgerald,Scott Fitzgerald (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Scott Fitzgerald,ScottFitzgerald.jpg,both
2,WMPID758,"TIFFANY, TOM",Tom Tiffany,WI,7.0,REP,Tom Tiffany,Tom Tiffany (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Tom Tiffany,Tom_Tiffany.jpg,both
3,WMPID2029,"ROE, ANN",Ann Roe,WI,1.0,DEM,Ann Roe,Ann Roe (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Ann Roe,Ann-Roe.PNG,both
4,WMPID861,"POCAN, MARK",Mark Pocan,WI,2.0,DEM,Mark Pocan,Mark Pocan (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mark Pocan,400px-Mark_Pocan_official_photo.jpg,both


In [1410]:
wi._merge.value_counts()

both          16
left_only      5
right_only     0
Name: _merge, dtype: int64

In [1411]:
wi.shape

(21, 12)

In [1376]:
WV = h.loc[h['cand_office_st']=='WV']

In [1377]:
WV.shape

(5, 6)

In [1378]:
WV

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
367,WMPID764,"WATSON, EUGENE LACY",Lacy Watson,WV,3.0,DEM
368,WMPID5094,"FOX-SPENCER, BELINDA",Belinda Fox-Spencer,WV,3.0,IND
3153,WMPID1615,"WENDELL, BARRY LEE MR.",Barry Lee Wendell,WV,2.0,DEM
3256,WMPID942,"MOONEY, ALEXANDER XAVIER",Alex Mooney,WV,2.0,REP
3612,WMPID1285,"MILLER, CAROL DEVINE",Carol Miller,WV,3.0,REP


In [1379]:
wv = fuzzy_merge(WV, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1380]:
wv

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
367,WMPID764,"WATSON, EUGENE LACY",Lacy Watson,WV,3.0,DEM,Lacy Watson
368,WMPID5094,"FOX-SPENCER, BELINDA",Belinda Fox-Spencer,WV,3.0,IND,Belinda Fox-Spencer
3153,WMPID1615,"WENDELL, BARRY LEE MR.",Barry Lee Wendell,WV,2.0,DEM,Barry Wendell
3256,WMPID942,"MOONEY, ALEXANDER XAVIER",Alex Mooney,WV,2.0,REP,
3612,WMPID1285,"MILLER, CAROL DEVINE",Carol Miller,WV,3.0,REP,Carol Miller


In [1381]:
wv = wv.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1382]:
wv.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID764,"WATSON, EUGENE LACY",Lacy Watson,WV,3.0,DEM,Lacy Watson,Lacy Watson (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Lacy Watson,LacyWatson.jpg,both
1,WMPID5094,"FOX-SPENCER, BELINDA",Belinda Fox-Spencer,WV,3.0,IND,Belinda Fox-Spencer,Belinda Fox-Spencer (Independent) \n\t\t\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Belinda Fox-Spencer,Belinda-FoxSpencer.PNG,both
2,WMPID1615,"WENDELL, BARRY LEE MR.",Barry Lee Wendell,WV,2.0,DEM,Barry Wendell,Barry Wendell (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Barry Wendell,Barry_Wendell.jpg,both
3,WMPID942,"MOONEY, ALEXANDER XAVIER",Alex Mooney,WV,2.0,REP,,NaN,NaN,NaN,NaN,left_only
4,WMPID1285,"MILLER, CAROL DEVINE",Carol Miller,WV,3.0,REP,Carol Miller,Carol Miller (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Carol Miller,Carol-Miller.PNG,both


In [1383]:
wv._merge.value_counts()

both          4
left_only     1
right_only    0
Name: _merge, dtype: int64

In [1384]:
wv.shape

(5, 12)

In [1351]:
WA = h.loc[h['cand_office_st']=='WA']

In [1352]:
WA.shape

(22, 6)

In [1353]:
WA

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
350,WMPID735,"LARSEN, RICK",Rick Larsen,WA,2.0,DEM
351,WMPID736,"CALL, JASON",Jason Call,WA,2.0,DEM
354,WMPID742,"KREISELMAIER, ELIZABETH",Elizabeth Kreiselmaier,WA,6.0,REP
356,WMPID743,"DELBENE, SUZAN K",Suzan DelBene,WA,1.0,DEM
360,WMPID746,"STRICKLAND, MARILYN",Marilyn Strickland,WA,10.0,DEM
3058,WMPID2621,"CAVALERI, VINCENT",Vincent Cavaleri,WA,1.0,REP
3059,WMPID1582,"MATTHEWS, DAN",Dan Matthews,WA,2.0,REP
3066,WMPID3363,"KENT, JOSEPH",Joseph Kent,WA,3.0,REP
3077,WMPID3256,"GLUESENKAMP PEREZ, MARIE",Marie Gluesenkamp Perez,WA,3.0,DEM
3084,WMPID3849,"WHITE, DOUG",Doug White,WA,4.0,DEM


In [1354]:
wa = fuzzy_merge(WA, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1355]:
wa

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
350,WMPID735,"LARSEN, RICK",Rick Larsen,WA,2.0,DEM,Rick Larsen
351,WMPID736,"CALL, JASON",Jason Call,WA,2.0,DEM,
354,WMPID742,"KREISELMAIER, ELIZABETH",Elizabeth Kreiselmaier,WA,6.0,REP,Elizabeth Kreiselmaier
356,WMPID743,"DELBENE, SUZAN K",Suzan DelBene,WA,1.0,DEM,Suzan DelBene
360,WMPID746,"STRICKLAND, MARILYN",Marilyn Strickland,WA,10.0,DEM,Marilyn Strickland
3058,WMPID2621,"CAVALERI, VINCENT",Vincent Cavaleri,WA,1.0,REP,Vincent Cavaleri
3059,WMPID1582,"MATTHEWS, DAN",Dan Matthews,WA,2.0,REP,Dan Matthews
3066,WMPID3363,"KENT, JOSEPH",Joseph Kent,WA,3.0,REP,
3077,WMPID3256,"GLUESENKAMP PEREZ, MARIE",Marie Gluesenkamp Perez,WA,3.0,DEM,Marie Gluesenkamp Pérez
3084,WMPID3849,"WHITE, DOUG",Doug White,WA,4.0,DEM,Doug White


In [1356]:
wa = wa.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1357]:
wa.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID735,"LARSEN, RICK",Rick Larsen,WA,2.0,DEM,Rick Larsen,Rick Larsen (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Rick Larsen,Rick_Larsen.jpg,both
1,WMPID736,"CALL, JASON",Jason Call,WA,2.0,DEM,,NaN,NaN,NaN,NaN,left_only
2,WMPID742,"KREISELMAIER, ELIZABETH",Elizabeth Kreiselmaier,WA,6.0,REP,Elizabeth Kreiselmaier,Elizabeth Kreiselmaier (R) \n\t\t\t\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,Elizabeth Kreiselmaier,Elizabeth-Kreiselmaier.PNG,both
3,WMPID743,"DELBENE, SUZAN K",Suzan DelBene,WA,1.0,DEM,Suzan DelBene,Suzan DelBene (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Suzan DelBene,Suzan_DelBene__official_portrait__112th_Congre...,both
4,WMPID746,"STRICKLAND, MARILYN",Marilyn Strickland,WA,10.0,DEM,Marilyn Strickland,Marilyn Strickland (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Marilyn Strickland,Marilyn-Strickland.jpg,both


In [1358]:
wa._merge.value_counts()

both          17
left_only      5
right_only     0
Name: _merge, dtype: int64

In [1359]:
wa.shape

(22, 12)

In [1326]:
VA = h.loc[h['cand_office_st']=='VA']

In [1327]:
VA.shape

(28, 6)

In [1328]:
VA

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
339,WMPID5138,"FOSTER, DAVID B.",David Foster,VA,1.0,IND
341,WMPID713,"BENJAMIN, LEON MR. SR.",Leon Benjamin,VA,4.0,REP
342,WMPID715,"GOOD, ROBERT G.",Bob Good,VA,5.0,REP
345,WMPID726,"GRIFFITH, H MORGAN",Morgan Griffith,VA,9.0,REP
2984,WMPID4367,"NICHOLS, AUSTIN RYAN MR",Austin Ryan Nichols,VA,1.0,IND
2987,WMPID3620,"JONES, HERBERT CARROLL JR",Herbert Carroll Jr Jones,VA,1.0,DEM
2989,WMPID2896,"KIGGANS, JENNIFER",Jennifer Kiggans,VA,2.0,REP
2992,WMPID1938,"NAMKUNG, TERRY TOLBERT MR.",Terry Tolbert Namkung,VA,3.0,REP
3000,WMPID2345,"THRONEBURG, JOSHUA",Joshua Throneburg,VA,5.0,DEM
3006,WMPID2756,"LEBEAU, DANIEL JOSEPH",Daniel Joseph Lebeau,VA,6.0,IND


In [1329]:
va = fuzzy_merge(VA, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1330]:
va

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
339,WMPID5138,"FOSTER, DAVID B.",David Foster,VA,1.0,IND,
341,WMPID713,"BENJAMIN, LEON MR. SR.",Leon Benjamin,VA,4.0,REP,Leon Benjamin Sr.
342,WMPID715,"GOOD, ROBERT G.",Bob Good,VA,5.0,REP,Bob Good
345,WMPID726,"GRIFFITH, H MORGAN",Morgan Griffith,VA,9.0,REP,H. Morgan Griffith
2984,WMPID4367,"NICHOLS, AUSTIN RYAN MR",Austin Ryan Nichols,VA,1.0,IND,
2987,WMPID3620,"JONES, HERBERT CARROLL JR",Herbert Carroll Jr Jones,VA,1.0,DEM,
2989,WMPID2896,"KIGGANS, JENNIFER",Jennifer Kiggans,VA,2.0,REP,Jennifer Kiggans
2992,WMPID1938,"NAMKUNG, TERRY TOLBERT MR.",Terry Tolbert Namkung,VA,3.0,REP,
3000,WMPID2345,"THRONEBURG, JOSHUA",Joshua Throneburg,VA,5.0,DEM,Joshua Throneburg
3006,WMPID2756,"LEBEAU, DANIEL JOSEPH",Daniel Joseph Lebeau,VA,6.0,IND,


In [1331]:
va = va.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1332]:
va.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID5138,"FOSTER, DAVID B.",David Foster,VA,1.0,IND,,NaN,NaN,NaN,NaN,left_only
1,WMPID713,"BENJAMIN, LEON MR. SR.",Leon Benjamin,VA,4.0,REP,Leon Benjamin Sr.,Leon Benjamin Sr. (R) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Leon Benjamin Sr.,Leon_Benjamin.jpeg,both
2,WMPID715,"GOOD, ROBERT G.",Bob Good,VA,5.0,REP,Bob Good,Bob Good (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Bob Good,Bob_Good.jpg,both
3,WMPID726,"GRIFFITH, H MORGAN",Morgan Griffith,VA,9.0,REP,H. Morgan Griffith,H. Morgan Griffith (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,H. Morgan Griffith,Morgan_Griffith.jpg,both
4,WMPID4367,"NICHOLS, AUSTIN RYAN MR",Austin Ryan Nichols,VA,1.0,IND,,NaN,NaN,NaN,NaN,left_only


In [1333]:
va._merge.value_counts()

both          15
left_only     13
right_only     0
Name: _merge, dtype: int64

In [1334]:
va.shape

(28, 12)

In [1301]:
UT = h.loc[h['cand_office_st']=='UT']

In [1302]:
UT.shape

(13, 6)

In [1303]:
UT

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
334,WMPID696,"MOORE, BLAKE",Blake Moore,UT,1.0,REP
336,WMPID706,"OWENS, BURGESS",Burgess Owens,UT,4.0,REP
2960,WMPID3504,"JONES, RICK",Rick Jones,UT,1.0,DEM
2961,WMPID857,"STEWART, CHRIS",Chris Stewart,UT,2.0,REP
2965,WMPID3382,"MCFARLAND, JAY JAYMAC",Jay Jaymac Mcfarland,UT,2.0,OTH
2967,WMPID2358,"MITCHELL, NICHOLAS",Nicholas Mitchell,UT,2.0,DEM
2968,WMPID3274,"EASLEY, CASSIE",Cassie Easley,UT,2.0,OTH
2974,WMPID4250,"WRIGHT, GLENN",Glenn Wright,UT,3.0,DEM
2976,WMPID2124,"STODDARD, MICHAEL",Michael Stoddard,UT,3.0,LIB
2980,WMPID2610,"WALKER, JANUARY",January Walker,UT,4.0,OTH


In [1304]:
ut = fuzzy_merge(UT, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1305]:
ut

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
334,WMPID696,"MOORE, BLAKE",Blake Moore,UT,1.0,REP,Blake Moore
336,WMPID706,"OWENS, BURGESS",Burgess Owens,UT,4.0,REP,Burgess Owens
2960,WMPID3504,"JONES, RICK",Rick Jones,UT,1.0,DEM,Rick Jones
2961,WMPID857,"STEWART, CHRIS",Chris Stewart,UT,2.0,REP,Chris Stewart
2965,WMPID3382,"MCFARLAND, JAY JAYMAC",Jay Jaymac Mcfarland,UT,2.0,OTH,
2967,WMPID2358,"MITCHELL, NICHOLAS",Nicholas Mitchell,UT,2.0,DEM,
2968,WMPID3274,"EASLEY, CASSIE",Cassie Easley,UT,2.0,OTH,Cassie Easley
2974,WMPID4250,"WRIGHT, GLENN",Glenn Wright,UT,3.0,DEM,Glenn J. Wright
2976,WMPID2124,"STODDARD, MICHAEL",Michael Stoddard,UT,3.0,LIB,Michael Stoddard
2980,WMPID2610,"WALKER, JANUARY",January Walker,UT,4.0,OTH,January Walker


In [1306]:
ut = ut.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1307]:
ut.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID696,"MOORE, BLAKE",Blake Moore,UT,1.0,REP,Blake Moore,Blake Moore (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Blake Moore,BlakeMoore.jpg,both
1,WMPID706,"OWENS, BURGESS",Burgess Owens,UT,4.0,REP,Burgess Owens,Burgess Owens (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Burgess Owens,Burgess-Owens.jpg,both
2,WMPID3504,"JONES, RICK",Rick Jones,UT,1.0,DEM,Rick Jones,Rick Jones (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Rick Jones,Rick-Jones-UT.PNG,both
3,WMPID857,"STEWART, CHRIS",Chris Stewart,UT,2.0,REP,Chris Stewart,Chris Stewart (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Chris Stewart,Chris-Stewart.jpg,both
4,WMPID3382,"MCFARLAND, JAY JAYMAC",Jay Jaymac Mcfarland,UT,2.0,OTH,,NaN,NaN,NaN,NaN,left_only


In [1308]:
ut._merge.value_counts()

both          11
left_only      2
right_only     0
Name: _merge, dtype: int64

In [1309]:
ut.shape

(13, 12)

In [1276]:
TX = h.loc[h['cand_office_st']=='TX']

In [1277]:
TX.shape

(92, 6)

In [1278]:
TX

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
298,WMPID628,"FALLON, PATRICK",Patrick Fallon,TX,4.0,REP
300,WMPID636,"HUNT, WESLEY",Wesley Hunt,TX,38.0,REP
302,WMPID2863,"JONES, LAURA",Laura Jones,TX,8.0,DEM
303,WMPID639,"TEAGUE, JOHNNY MARK DR.",Johnny Teague,TX,7.0,REP
305,WMPID3680,"ERIKSEN, ROY",Roy Eriksen,TX,8.0,LIB
...,...,...,...,...,...,...
3582,WMPID1259,"CLOUD, MICHAEL",Michael Cloud,TX,27.0,REP
3585,WMPID2797,"IDROGO, MICHAEL",Michael Idrogo,TX,35.0,LIB
3586,WMPID1260,"LEONE, ROSS LYNN",Ross Leone,TX,15.0,LIB
3587,WMPID1261,"GARCIA, SYLVIA R.",Sylvia Garcia,TX,29.0,DEM


In [1279]:
tx = fuzzy_merge(TX, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1280]:
tx

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
298,WMPID628,"FALLON, PATRICK",Patrick Fallon,TX,4.0,REP,
300,WMPID636,"HUNT, WESLEY",Wesley Hunt,TX,38.0,REP,Wesley Hunt
302,WMPID2863,"JONES, LAURA",Laura Jones,TX,8.0,DEM,Laura Jones
303,WMPID639,"TEAGUE, JOHNNY MARK DR.",Johnny Teague,TX,7.0,REP,Johnny Teague
305,WMPID3680,"ERIKSEN, ROY",Roy Eriksen,TX,8.0,LIB,Roy Eriksen
...,...,...,...,...,...,...,...
3582,WMPID1259,"CLOUD, MICHAEL",Michael Cloud,TX,27.0,REP,Michael Cloud
3585,WMPID2797,"IDROGO, MICHAEL",Michael Idrogo,TX,35.0,LIB,
3586,WMPID1260,"LEONE, ROSS LYNN",Ross Leone,TX,15.0,LIB,
3587,WMPID1261,"GARCIA, SYLVIA R.",Sylvia Garcia,TX,29.0,DEM,Sylvia Garcia


In [1281]:
tx = tx.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1282]:
tx.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID628,"FALLON, PATRICK",Patrick Fallon,TX,4.0,REP,,NaN,NaN,NaN,NaN,left_only
1,WMPID636,"HUNT, WESLEY",Wesley Hunt,TX,38.0,REP,Wesley Hunt,Wesley Hunt (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Wesley Hunt,Wesley-Hunt.jpg,both
2,WMPID2863,"JONES, LAURA",Laura Jones,TX,8.0,DEM,Laura Jones,Laura Jones (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Laura Jones,Laura-Jones.jpg,both
3,WMPID639,"TEAGUE, JOHNNY MARK DR.",Johnny Teague,TX,7.0,REP,Johnny Teague,Johnny Teague (R) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Johnny Teague,jt.png,both
4,WMPID3680,"ERIKSEN, ROY",Roy Eriksen,TX,8.0,LIB,Roy Eriksen,Roy Eriksen (L) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Roy Eriksen,Roy_Eriksen.jpg,both


In [1283]:
tx._merge.value_counts()

both          67
left_only     25
right_only     0
Name: _merge, dtype: int64

In [1284]:
tx.shape

(92, 12)

In [1249]:
TN = h.loc[h['cand_office_st']=='TN']

In [1250]:
TN.shape

(33, 6)

In [1251]:
TN

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
288,WMPID606,"HARSHBARGER, DIANA",Diana Harshbarger,TN,1.0,REP
289,WMPID5137,"BAKER, RICHARD",Richard Baker,TN,1.0,IND
290,WMPID607,"FLEISCHMANN, CHARLES J",Chuck Fleischmann,TN,3.0,REP
291,WMPID609,"GORMAN, MEG",Meg Gorman,TN,3.0,DEM
292,WMPID610,"DESJARLAIS, SCOTT HON.",Scott Desjarlais,TN,4.0,REP
293,WMPID2077,"FLINN, GEORGE S DR JR",George Jr Flinn,TN,9.0,IND
294,WMPID1239,"BERGMANN, CHARLOTTE",Charlotte Bergmann,TN,9.0,REP
295,WMPID617,"CLARK, DENNIS JEFFREY",Dennis Clark,TN,9.0,IND
2635,WMPID4286,"PARSONS, CAMERON",Cameron Parsons,TN,1.0,DEM
2637,WMPID5023,"MAKROM, AHMED MATTHEW",Ahmed Matthew Makrom,TN,1.0,IND


In [1252]:
tn = fuzzy_merge(TN, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1253]:
tn

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
288,WMPID606,"HARSHBARGER, DIANA",Diana Harshbarger,TN,1.0,REP,Diana Harshbarger
289,WMPID5137,"BAKER, RICHARD",Richard Baker,TN,1.0,IND,Richard Baker
290,WMPID607,"FLEISCHMANN, CHARLES J",Chuck Fleischmann,TN,3.0,REP,
291,WMPID609,"GORMAN, MEG",Meg Gorman,TN,3.0,DEM,Meg Gorman
292,WMPID610,"DESJARLAIS, SCOTT HON.",Scott Desjarlais,TN,4.0,REP,Scott DesJarlais
293,WMPID2077,"FLINN, GEORGE S DR JR",George Jr Flinn,TN,9.0,IND,George Flinn Jr.
294,WMPID1239,"BERGMANN, CHARLOTTE",Charlotte Bergmann,TN,9.0,REP,Charlotte Bergmann
295,WMPID617,"CLARK, DENNIS JEFFREY",Dennis Clark,TN,9.0,IND,Dennis Clark
2635,WMPID4286,"PARSONS, CAMERON",Cameron Parsons,TN,1.0,DEM,Cameron Parsons
2637,WMPID5023,"MAKROM, AHMED MATTHEW",Ahmed Matthew Makrom,TN,1.0,IND,


In [1254]:
tn = tn.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1255]:
tn.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID606,"HARSHBARGER, DIANA",Diana Harshbarger,TN,1.0,REP,Diana Harshbarger,Diana Harshbarger (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Diana Harshbarger,_Diana-Harshbarger_.PNG,both
1,WMPID5137,"BAKER, RICHARD",Richard Baker,TN,1.0,IND,Richard Baker,Richard Baker (Independent) \n\t\t\t\t\t\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Richard Baker,Matt_Makrom.jpeg,both
2,WMPID607,"FLEISCHMANN, CHARLES J",Chuck Fleischmann,TN,3.0,REP,,NaN,NaN,NaN,NaN,left_only
3,WMPID609,"GORMAN, MEG",Meg Gorman,TN,3.0,DEM,Meg Gorman,Meg Gorman (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Meg Gorman,Meg_Gorman.PNG,both
4,WMPID610,"DESJARLAIS, SCOTT HON.",Scott Desjarlais,TN,4.0,REP,Scott DesJarlais,Scott DesJarlais (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Scott DesJarlais,Scott_DesJarlais.jpg,both


In [1256]:
tn._merge.value_counts()

both          25
left_only      8
right_only     0
Name: _merge, dtype: int64

In [1257]:
tn.shape

(33, 12)

In [1224]:
SC = h.loc[h['cand_office_st']=='SC']

In [1225]:
SC.shape

(18, 6)

In [1226]:
SC

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
286,WMPID595,"MACE, NANCY",Nancy Mace,SC,1.0,REP
287,WMPID598,"DUNCAN, JEFFREY D MR.",Jeff Duncan,SC,3.0,REP
371,WMPID2813,"BURKS, COURTNEY YVONNE",Courtney Yvonne Burks,SC,1.0,IND
2589,WMPID4171,"ANDREWS, ANNIE DR.",Annie Andrews,SC,1.0,DEM
2593,WMPID839,"CLYBURN, JAMES E.",James E Clyburn,SC,6.0,DEM
2594,WMPID840,"WILSON, JOE THE HON.",Joe Wilson,SC,2.0,REP
2597,WMPID4645,"LARKINS, JUDDSON",Juddson Larkins,SC,2.0,DEM
2598,WMPID5111,"ODDO, JOSEPH",Joseph Oddo,SC,1.0,OTH
2599,WMPID841,"CHANDLER, MICHAEL",Michael Chandler,SC,4.0,CON
2604,WMPID4826,"HILL, KEN",Ken Hill,SC,4.0,DEM


In [1227]:
sc = fuzzy_merge(SC, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1228]:
sc

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
286,WMPID595,"MACE, NANCY",Nancy Mace,SC,1.0,REP,Nancy Mace
287,WMPID598,"DUNCAN, JEFFREY D MR.",Jeff Duncan,SC,3.0,REP,Jeff Duncan
371,WMPID2813,"BURKS, COURTNEY YVONNE",Courtney Yvonne Burks,SC,1.0,IND,
2589,WMPID4171,"ANDREWS, ANNIE DR.",Annie Andrews,SC,1.0,DEM,Annie Andrews
2593,WMPID839,"CLYBURN, JAMES E.",James E Clyburn,SC,6.0,DEM,James Clyburn
2594,WMPID840,"WILSON, JOE THE HON.",Joe Wilson,SC,2.0,REP,Joe Wilson
2597,WMPID4645,"LARKINS, JUDDSON",Juddson Larkins,SC,2.0,DEM,
2598,WMPID5111,"ODDO, JOSEPH",Joseph Oddo,SC,1.0,OTH,Joseph Oddo
2599,WMPID841,"CHANDLER, MICHAEL",Michael Chandler,SC,4.0,CON,
2604,WMPID4826,"HILL, KEN",Ken Hill,SC,4.0,DEM,


In [1229]:
sc = sc.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1230]:
sc.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID595,"MACE, NANCY",Nancy Mace,SC,1.0,REP,Nancy Mace,Nancy Mace (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Nancy Mace,_Nancy_Mace_.PNG,both
1,WMPID598,"DUNCAN, JEFFREY D MR.",Jeff Duncan,SC,3.0,REP,Jeff Duncan,Jeff Duncan (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Jeff Duncan,Jeff_Duncan.jpg,both
2,WMPID2813,"BURKS, COURTNEY YVONNE",Courtney Yvonne Burks,SC,1.0,IND,,NaN,NaN,NaN,NaN,left_only
3,WMPID4171,"ANDREWS, ANNIE DR.",Annie Andrews,SC,1.0,DEM,Annie Andrews,Annie Andrews (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Annie Andrews,Annieandrews.jpg,both
4,WMPID839,"CLYBURN, JAMES E.",James E Clyburn,SC,6.0,DEM,James Clyburn,James Clyburn (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,James Clyburn,James_Clyburn.jpg,both


In [1231]:
sc._merge.value_counts()

both          13
left_only      5
right_only     0
Name: _merge, dtype: int64

In [1232]:
sc.shape

(18, 12)

In [1439]:
RI = h.loc[h['cand_office_st']=='RI']

In [1440]:
RI.shape

(5, 6)

In [1441]:
RI

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
283,WMPID590,"CICILLINE, DAVID N",David Cicilline,RI,1.0,DEM
2570,WMPID1421,"WATERS, ALLEN R",Allen Waters,RI,1.0,REP
2575,WMPID3071,"MAGAZINER, SETH",Seth Magaziner,RI,2.0,DEM
2579,WMPID4637,"FUNG, ALLAN",Allan Fung,RI,2.0,REP
2582,WMPID4956,"GILBERT, WILLIAM",William Gilbert,RI,2.0,UN


In [1442]:
ri = fuzzy_merge(RI, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1443]:
ri

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
283,WMPID590,"CICILLINE, DAVID N",David Cicilline,RI,1.0,DEM,David N. Cicilline
2570,WMPID1421,"WATERS, ALLEN R",Allen Waters,RI,1.0,REP,Allen Waters
2575,WMPID3071,"MAGAZINER, SETH",Seth Magaziner,RI,2.0,DEM,Seth Magaziner
2579,WMPID4637,"FUNG, ALLAN",Allan Fung,RI,2.0,REP,Allan Fung
2582,WMPID4956,"GILBERT, WILLIAM",William Gilbert,RI,2.0,UN,


In [1444]:
ri = ri.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1445]:
ri.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID590,"CICILLINE, DAVID N",David Cicilline,RI,1.0,DEM,David N. Cicilline,David N. Cicilline (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,David N. Cicilline,David_CicillineRI.jpg,both
1,WMPID1421,"WATERS, ALLEN R",Allen Waters,RI,1.0,REP,Allen Waters,Allen Waters (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Allen Waters,Allen_Waters_01.jpg,both
2,WMPID3071,"MAGAZINER, SETH",Seth Magaziner,RI,2.0,DEM,Seth Magaziner,Seth Magaziner (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Seth Magaziner,Seth-Magaziner.PNG,both
3,WMPID4637,"FUNG, ALLAN",Allan Fung,RI,2.0,REP,Allan Fung,Allan Fung (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Allan Fung,Allan_Fung.jpg,both
4,WMPID4956,"GILBERT, WILLIAM",William Gilbert,RI,2.0,UN,,NaN,NaN,NaN,NaN,left_only


In [1446]:
ri._merge.value_counts()

both          4
left_only     1
right_only    0
Name: _merge, dtype: int64

In [1447]:
ri.shape

(5, 12)

In [1183]:
PA = h.loc[h['cand_office_st']=='PA']

In [1184]:
PA.shape

(35, 6)

In [1185]:
PA

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
272,WMPID564,"KELLY, GEORGE J JR",Mike Kelly,PA,16.0,REP
275,WMPID571,"SCHELLER, LISA",Lisa Scheller,PA,7.0,REP
277,WMPID572,"BOGNET, JIM",Jim Bognet,PA,8.0,REP
2500,WMPID3124,"CONOLY, HENRY",Henry Conoly,PA,1.0,GRE
2501,WMPID4502,"EHASZ, ASHLEY",Ashley Ehasz,PA,1.0,DEM
2504,WMPID1701,"AVERY, CAROLINE",Caroline Avery,PA,1.0,LIB
2509,WMPID3576,"BASHIR, HAROON",Haroon Bashir,PA,2.0,REP
2514,WMPID837,"PERRY, SCOTT",Scott Perry,PA,10.0,REP
2515,WMPID2876,"NASCIMENTO, CHRISTIAN",Christian Nascimento,PA,4.0,REP
2517,WMPID3975,"GALLUCH, DAVID",David Galluch,PA,5.0,REP


In [1186]:
pa = fuzzy_merge(PA, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1187]:
pa

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
272,WMPID564,"KELLY, GEORGE J JR",Mike Kelly,PA,16.0,REP,Mike Kelly
275,WMPID571,"SCHELLER, LISA",Lisa Scheller,PA,7.0,REP,Lisa Scheller
277,WMPID572,"BOGNET, JIM",Jim Bognet,PA,8.0,REP,Jim Bognet
2500,WMPID3124,"CONOLY, HENRY",Henry Conoly,PA,1.0,GRE,Henry Conoly
2501,WMPID4502,"EHASZ, ASHLEY",Ashley Ehasz,PA,1.0,DEM,Ashley Ehasz
2504,WMPID1701,"AVERY, CAROLINE",Caroline Avery,PA,1.0,LIB,Caroline Avery
2509,WMPID3576,"BASHIR, HAROON",Haroon Bashir,PA,2.0,REP,
2514,WMPID837,"PERRY, SCOTT",Scott Perry,PA,10.0,REP,Scott Perry
2515,WMPID2876,"NASCIMENTO, CHRISTIAN",Christian Nascimento,PA,4.0,REP,Christian Nascimento
2517,WMPID3975,"GALLUCH, DAVID",David Galluch,PA,5.0,REP,David Galluch


In [1188]:
pa = pa.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1189]:
pa.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID564,"KELLY, GEORGE J JR",Mike Kelly,PA,16.0,REP,Mike Kelly,Mike Kelly (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mike Kelly,Mike_Kelly__Official_Portrait__112th_Congress-...,both
1,WMPID571,"SCHELLER, LISA",Lisa Scheller,PA,7.0,REP,Lisa Scheller,Lisa Scheller (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Lisa Scheller,Scheller.jpg,both
2,WMPID572,"BOGNET, JIM",Jim Bognet,PA,8.0,REP,Jim Bognet,Jim Bognet (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Jim Bognet,JimBognet.png,both
3,WMPID3124,"CONOLY, HENRY",Henry Conoly,PA,1.0,GRE,Henry Conoly,Henry Conoly (G) (Write-in) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Henry Conoly,Caroline_Avery.png,both
4,WMPID4502,"EHASZ, ASHLEY",Ashley Ehasz,PA,1.0,DEM,Ashley Ehasz,Ashley Ehasz (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Ashley Ehasz,Ashley-Ehasz.PNG,both


In [1190]:
pa._merge.value_counts()

both          31
left_only      4
right_only     0
Name: _merge, dtype: int64

In [1191]:
pa.shape

(35, 12)

In [1158]:
OR = h.loc[h['cand_office_st']=='OR']

In [1159]:
OR.shape

(18, 6)

In [1160]:
OR

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
265,WMPID555,"BENTZ, CLIFF",Cliff Bentz,OR,2.0,REP
267,WMPID559,"HARBOUR, JOANNA",Joanna Harbour,OR,3.0,REP
268,WMPID560,"SKARLATOS, ALEK",Alek Skarlatos,OR,4.0,REP
2451,WMPID836,"BONAMICI, SUZANNE MS.",Suzanne Bonamici,OR,1.0,DEM
2454,WMPID4638,"RASCON, NICHOLAS",Nicholas Rascon,OR,1.0,OTH
2456,WMPID4232,"MANN, CHRISTOPHER ALLAN",Christopher Allan Mann,OR,1.0,REP
2459,WMPID3110,"YETTER, JOSEPH FREDERICK III",Joseph Frederick III Yetter,OR,2.0,DEM
2465,WMPID4342,"HOYLE, VALERIE",Valerie Hoyle,OR,4.0,DEM
2476,WMPID5119,"LEATHERBERRY, LEVI",Levi Leatherberry,OR,4.0,LIB
2479,WMPID3896,"CHAVEZ-DEREMER, LORI",Lori Chavez-Deremer,OR,5.0,REP


In [1161]:
xor = fuzzy_merge(OR, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1162]:
xor

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
265,WMPID555,"BENTZ, CLIFF",Cliff Bentz,OR,2.0,REP,Cliff Bentz
267,WMPID559,"HARBOUR, JOANNA",Joanna Harbour,OR,3.0,REP,Joanna Harbour
268,WMPID560,"SKARLATOS, ALEK",Alek Skarlatos,OR,4.0,REP,Alek Skarlatos
2451,WMPID836,"BONAMICI, SUZANNE MS.",Suzanne Bonamici,OR,1.0,DEM,Suzanne Bonamici
2454,WMPID4638,"RASCON, NICHOLAS",Nicholas Rascon,OR,1.0,OTH,
2456,WMPID4232,"MANN, CHRISTOPHER ALLAN",Christopher Allan Mann,OR,1.0,REP,
2459,WMPID3110,"YETTER, JOSEPH FREDERICK III",Joseph Frederick III Yetter,OR,2.0,DEM,
2465,WMPID4342,"HOYLE, VALERIE",Valerie Hoyle,OR,4.0,DEM,
2476,WMPID5119,"LEATHERBERRY, LEVI",Levi Leatherberry,OR,4.0,LIB,Levi Leatherberry
2479,WMPID3896,"CHAVEZ-DEREMER, LORI",Lori Chavez-Deremer,OR,5.0,REP,Lori Chavez-DeRemer


In [1163]:
xor = xor.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1164]:
xor.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID555,"BENTZ, CLIFF",Cliff Bentz,OR,2.0,REP,Cliff Bentz,Cliff Bentz (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Cliff Bentz,CliffBentz.jpg,both
1,WMPID559,"HARBOUR, JOANNA",Joanna Harbour,OR,3.0,REP,Joanna Harbour,Joanna Harbour (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Joanna Harbour,Joanna-Harbour.PNG,both
2,WMPID560,"SKARLATOS, ALEK",Alek Skarlatos,OR,4.0,REP,Alek Skarlatos,Alek Skarlatos (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Alek Skarlatos,Alek_Skarlatos.jpg,both
3,WMPID836,"BONAMICI, SUZANNE MS.",Suzanne Bonamici,OR,1.0,DEM,Suzanne Bonamici,Suzanne Bonamici (D / Working Families Party) ...,https://s3.amazonaws.com/ballotpedia-api4/file...,Suzanne Bonamici,Suzanne_Bonamici.jpg,both
4,WMPID4638,"RASCON, NICHOLAS",Nicholas Rascon,OR,1.0,OTH,,NaN,NaN,NaN,NaN,left_only


In [1165]:
xor._merge.value_counts()

both          13
left_only      5
right_only     0
Name: _merge, dtype: int64

In [1166]:
xor.shape

(18, 12)

In [1133]:
OK = h.loc[h['cand_office_st']=='OK']

In [1134]:
OK.shape

(14, 6)

In [1135]:
OK

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
263,WMPID551,"BICE, STEPHANIE",Stephanie Bice,OK,5.0,REP
2423,WMPID4392,"MARTIN, ADAM",Adam Martin,OK,1.0,DEM
2431,WMPID4465,"FRIX, AVERY",Avery Frix,OK,2.0,REP
2435,WMPID3542,"ANDREWS, NAOMI",Naomi Andrews,OK,2.0,DEM
2436,WMPID3085,"BRECHEEN, JOSH",Josh Brecheen,OK,2.0,REP
2438,WMPID4923,"ROBINSON, BULLDOG BEN",Bulldog Ben Robinson,OK,2.0,IND
2441,WMPID4920,"ROSS, JEREMIAH A",Jeremiah Ross,OK,3.0,DEM
2443,WMPID835,"COLE, TOM",Tom Cole,OK,4.0,REP
2447,WMPID3077,"HARRIS-TILL, JOSHUA",Joshua Harris-Till,OK,5.0,DEM
2449,WMPID4909,"FROSCH, DAVID K.",David Frosch,OK,5.0,IND


In [1136]:
ok = fuzzy_merge(OK, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1137]:
ok

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
263,WMPID551,"BICE, STEPHANIE",Stephanie Bice,OK,5.0,REP,Stephanie Bice
2423,WMPID4392,"MARTIN, ADAM",Adam Martin,OK,1.0,DEM,Adam Martin
2431,WMPID4465,"FRIX, AVERY",Avery Frix,OK,2.0,REP,
2435,WMPID3542,"ANDREWS, NAOMI",Naomi Andrews,OK,2.0,DEM,Naomi Andrews
2436,WMPID3085,"BRECHEEN, JOSH",Josh Brecheen,OK,2.0,REP,Josh Brecheen
2438,WMPID4923,"ROBINSON, BULLDOG BEN",Bulldog Ben Robinson,OK,2.0,IND,Ben Robinson
2441,WMPID4920,"ROSS, JEREMIAH A",Jeremiah Ross,OK,3.0,DEM,Jeremiah Ross
2443,WMPID835,"COLE, TOM",Tom Cole,OK,4.0,REP,Tom Cole
2447,WMPID3077,"HARRIS-TILL, JOSHUA",Joshua Harris-Till,OK,5.0,DEM,Joshua Harris-Till
2449,WMPID4909,"FROSCH, DAVID K.",David Frosch,OK,5.0,IND,David Frosch


In [1138]:
ok = ok.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1139]:
ok.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID551,"BICE, STEPHANIE",Stephanie Bice,OK,5.0,REP,Stephanie Bice,Stephanie Bice (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Stephanie Bice,Stephanie_Bice.jpg,both
1,WMPID4392,"MARTIN, ADAM",Adam Martin,OK,1.0,DEM,Adam Martin,Adam Martin (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Adam Martin,Adam_Martin.jpeg,both
2,WMPID4465,"FRIX, AVERY",Avery Frix,OK,2.0,REP,,NaN,NaN,NaN,NaN,left_only
3,WMPID3542,"ANDREWS, NAOMI",Naomi Andrews,OK,2.0,DEM,Naomi Andrews,Naomi Andrews (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Naomi Andrews,Naomi-Andrews.PNG,both
4,WMPID3085,"BRECHEEN, JOSH",Josh Brecheen,OK,2.0,REP,Josh Brecheen,Josh Brecheen (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Josh Brecheen,Josh-Brecheen.PNG,both


In [1140]:
ok._merge.value_counts()

both          13
left_only      1
right_only     0
Name: _merge, dtype: int64

In [1141]:
ok.shape

(14, 12)

In [1108]:
OH = h.loc[h['cand_office_st']=='OH']

In [1109]:
OH.shape

(33, 6)

In [1110]:
OH

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
252,WMPID534,"JOHNSON, BILL",Bill Johnson,OH,6.0,REP
2329,WMPID3309,"LANDSMAN, GREG",Greg Landsman,OH,1.0,DEM
2332,WMPID828,"WENSTRUP, BRAD DR.",Brad Wenstrup,OH,2.0,REP
2333,WMPID1563,"MEADOWS, SAMANTHA",Samantha Meadows,OH,2.0,DEM
2336,WMPID829,"TURNER, MICHAEL R REP.",Michael R Turner,OH,10.0,REP
2337,WMPID830,"BEATTY, JOYCE",Joyce Beatty,OH,3.0,DEM
2340,WMPID3362,"STAHLEY, LEE R.",Lee Stahley,OH,3.0,REP
2341,WMPID2629,"WILSON, TAMIE",Tamie Wilson,OH,4.0,DEM
2343,WMPID3174,"BADDAR, YOUSEFF MILAD",Youseff Milad Baddar,OH,9.0,IND
2345,WMPID3698,"SWARTZ, CRAIG STEPHEN",Craig Stephen Swartz,OH,5.0,DEM


In [1111]:
oh = fuzzy_merge(OH, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1112]:
oh

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
252,WMPID534,"JOHNSON, BILL",Bill Johnson,OH,6.0,REP,Bill Johnson
2329,WMPID3309,"LANDSMAN, GREG",Greg Landsman,OH,1.0,DEM,Greg Landsman
2332,WMPID828,"WENSTRUP, BRAD DR.",Brad Wenstrup,OH,2.0,REP,Brad Wenstrup
2333,WMPID1563,"MEADOWS, SAMANTHA",Samantha Meadows,OH,2.0,DEM,Samantha Meadows
2336,WMPID829,"TURNER, MICHAEL R REP.",Michael R Turner,OH,10.0,REP,Michael Turner
2337,WMPID830,"BEATTY, JOYCE",Joyce Beatty,OH,3.0,DEM,Joyce Beatty
2340,WMPID3362,"STAHLEY, LEE R.",Lee Stahley,OH,3.0,REP,Lee Stahley
2341,WMPID2629,"WILSON, TAMIE",Tamie Wilson,OH,4.0,DEM,Tamie Wilson
2343,WMPID3174,"BADDAR, YOUSEFF MILAD",Youseff Milad Baddar,OH,9.0,IND,
2345,WMPID3698,"SWARTZ, CRAIG STEPHEN",Craig Stephen Swartz,OH,5.0,DEM,


In [1113]:
oh = oh.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1114]:
oh.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID534,"JOHNSON, BILL",Bill Johnson,OH,6.0,REP,Bill Johnson,Bill Johnson (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Bill Johnson,Bill_Johnson.jpg,both
1,WMPID3309,"LANDSMAN, GREG",Greg Landsman,OH,1.0,DEM,Greg Landsman,Greg Landsman (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Greg Landsman,Greg_Landsman.jpeg,both
2,WMPID828,"WENSTRUP, BRAD DR.",Brad Wenstrup,OH,2.0,REP,Brad Wenstrup,Brad Wenstrup (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Brad Wenstrup,Brad_Wenstrup.jpg,both
3,WMPID1563,"MEADOWS, SAMANTHA",Samantha Meadows,OH,2.0,DEM,Samantha Meadows,Samantha Meadows (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Samantha Meadows,smeadows.jpg,both
4,WMPID829,"TURNER, MICHAEL R REP.",Michael R Turner,OH,10.0,REP,Michael Turner,Michael Turner (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Michael Turner,Mike_Turner_OH-7_fixed.jpg,both


In [1115]:
oh._merge.value_counts()

both          26
left_only      7
right_only     0
Name: _merge, dtype: int64

In [1116]:
oh.shape

(33, 12)

In [1083]:
NC = h.loc[h['cand_office_st']=='NC']

In [1084]:
NC.shape

(30, 6)

In [1085]:
NC

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
197,WMPID402,"ROSS, DEBORAH",Deborah Ross,NC,2.0,DEM
199,WMPID405,"SMITH, SANDY",Sandy Smith,NC,1.0,REP
200,WMPID406,"MURPHY, GREGORY FRANCIS DR.",Greg Murphy,NC,3.0,REP
205,WMPID413,"BISHOP, JAMES DANIEL",Dan Bishop,NC,8.0,REP
1855,WMPID813,"PATEL, ESHAN",Eshan Patel,NC,1.0,IND
1868,WMPID1782,"DAVIS, DON",Don Davis,NC,1.0,DEM
1872,WMPID4687,"GASKINS, BARBARA D MS.",Barbara Gaskins,NC,3.0,DEM
1884,WMPID2787,"CLARK, ROBERT BENJAMIN III",Robert Benjamin III Clark,NC,4.0,DEM
1890,WMPID3526,"HINES, ROBERT",Robert Hines,NC,13.0,REP
1895,WMPID3128,"FOUSHEE, VALERIE",Valerie Foushee,NC,4.0,DEM


In [1086]:
nc = fuzzy_merge(NC, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1087]:
nc

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
197,WMPID402,"ROSS, DEBORAH",Deborah Ross,NC,2.0,DEM,Deborah Ross
199,WMPID405,"SMITH, SANDY",Sandy Smith,NC,1.0,REP,Sandy Smith
200,WMPID406,"MURPHY, GREGORY FRANCIS DR.",Greg Murphy,NC,3.0,REP,
205,WMPID413,"BISHOP, JAMES DANIEL",Dan Bishop,NC,8.0,REP,Dan Bishop
1855,WMPID813,"PATEL, ESHAN",Eshan Patel,NC,1.0,IND,
1868,WMPID1782,"DAVIS, DON",Don Davis,NC,1.0,DEM,
1872,WMPID4687,"GASKINS, BARBARA D MS.",Barbara Gaskins,NC,3.0,DEM,Barbara Gaskins
1884,WMPID2787,"CLARK, ROBERT BENJAMIN III",Robert Benjamin III Clark,NC,4.0,DEM,
1890,WMPID3526,"HINES, ROBERT",Robert Hines,NC,13.0,REP,
1895,WMPID3128,"FOUSHEE, VALERIE",Valerie Foushee,NC,4.0,DEM,Valerie Foushee


In [1088]:
nc = nc.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1089]:
nc.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID402,"ROSS, DEBORAH",Deborah Ross,NC,2.0,DEM,Deborah Ross,Deborah Ross (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Deborah Ross,Deborah-Ross.jpg,both
1,WMPID405,"SMITH, SANDY",Sandy Smith,NC,1.0,REP,Sandy Smith,Sandy Smith (R) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Sandy Smith,ssmith.JPG,both
2,WMPID406,"MURPHY, GREGORY FRANCIS DR.",Greg Murphy,NC,3.0,REP,,NaN,NaN,NaN,NaN,left_only
3,WMPID413,"BISHOP, JAMES DANIEL",Dan Bishop,NC,8.0,REP,Dan Bishop,Dan Bishop (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Dan Bishop,Dan-Bishop.PNG,both
4,WMPID813,"PATEL, ESHAN",Eshan Patel,NC,1.0,IND,,NaN,NaN,NaN,NaN,left_only


In [1090]:
nc._merge.value_counts()

both          25
left_only      5
right_only     0
Name: _merge, dtype: int64

In [1091]:
nc.shape

(30, 12)

In [1043]:
NY = h.loc[h['cand_office_st']=='NY']

In [1044]:
NY.shape

(113, 6)

In [1045]:
NY

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
230,WMPID2343,"FLEMING, BRIDGET",Bridget Fleming,NY,1.0,DEM
231,WMPID470,"GORDON, JACQUELINE",Jackie Gordon,NY,2.0,DEM
232,WMPID471,"GARBARINO, ANDREW",Andrew Garbarino,NY,2.0,REP
233,WMPID2364,"D'ARRIGO, MELANIE",Melanie D'Arrigo,NY,3.0,DEM
234,WMPID475,"DEVOLDER-SANTOS, GEORGE ANTHONY",George Devolder-Santos,NY,3.0,REP
...,...,...,...,...,...,...
3523,WMPID2619,"RYAN, PATRICK",Patrick Ryan,NY,18.0,DEM
3524,WMPID1196,"TONKO, PAUL DAVID",Paul Tonko,NY,20.0,DEM
3526,WMPID1629,"RILEY, JOSH",Josh Riley,NY,19.0,DEM
3527,WMPID3594,"DELLA PIA, MAX",Max Della Pia,NY,23.0,DEM


In [1046]:
ny = fuzzy_merge(NY, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1047]:
ny

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
230,WMPID2343,"FLEMING, BRIDGET",Bridget Fleming,NY,1.0,DEM,Bridget M. Fleming
231,WMPID470,"GORDON, JACQUELINE",Jackie Gordon,NY,2.0,DEM,Jackie Gordon
232,WMPID471,"GARBARINO, ANDREW",Andrew Garbarino,NY,2.0,REP,Andrew Garbarino
233,WMPID2364,"D'ARRIGO, MELANIE",Melanie D'Arrigo,NY,3.0,DEM,
234,WMPID475,"DEVOLDER-SANTOS, GEORGE ANTHONY",George Devolder-Santos,NY,3.0,REP,George Devolder-Santos
...,...,...,...,...,...,...,...
3523,WMPID2619,"RYAN, PATRICK",Patrick Ryan,NY,18.0,DEM,
3524,WMPID1196,"TONKO, PAUL DAVID",Paul Tonko,NY,20.0,DEM,Paul Tonko
3526,WMPID1629,"RILEY, JOSH",Josh Riley,NY,19.0,DEM,Josh Riley
3527,WMPID3594,"DELLA PIA, MAX",Max Della Pia,NY,23.0,DEM,Max Della Pia


In [1048]:
ny = ny.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1049]:
ny.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID2343,"FLEMING, BRIDGET",Bridget Fleming,NY,1.0,DEM,Bridget M. Fleming,Bridget M. Fleming (D / Working Families Party...,https://s3.amazonaws.com/ballotpedia-api4/file...,Bridget M. Fleming,Bridget-Fleming.PNG,both
1,WMPID470,"GORDON, JACQUELINE",Jackie Gordon,NY,2.0,DEM,Jackie Gordon,Jackie Gordon (D / Working Families Party) \n...,https://s3.amazonaws.com/ballotpedia-api4/file...,Jackie Gordon,JackieGordon.jpg,both
2,WMPID471,"GARBARINO, ANDREW",Andrew Garbarino,NY,2.0,REP,Andrew Garbarino,Andrew Garbarino (R / Conservative Party) \n\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Andrew Garbarino,Andrew-Garbarino.PNG,both
3,WMPID2364,"D'ARRIGO, MELANIE",Melanie D'Arrigo,NY,3.0,DEM,,NaN,NaN,NaN,NaN,left_only
4,WMPID475,"DEVOLDER-SANTOS, GEORGE ANTHONY",George Devolder-Santos,NY,3.0,REP,George Devolder-Santos,George Devolder-Santos (R / Conservative Party...,https://s3.amazonaws.com/ballotpedia-api4/file...,George Devolder-Santos,George_Santos2022.jpg,both


In [1050]:
ny._merge.value_counts()

both          61
left_only     52
right_only     0
Name: _merge, dtype: int64

In [1051]:
ny.shape

(113, 12)

In [1477]:
NM = h.loc[h['cand_office_st']=='NM']

In [1478]:
NM.shape

(7, 6)

In [1479]:
NM

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
225,WMPID451,"GARCIA HOLMES, MICHELLE",Michelle Garcia Holmes,NM,1.0,REP
226,WMPID453,"LEGER FERNANDEZ, TERESA",Teresa Leger,NM,3.0,DEM
2093,WMPID3359,"STANSBURY, MELANIE",Melanie Stansbury,NM,1.0,DEM
2109,WMPID4714,"CHICK, CAMERON ALTON MR. SR",Cameron Alton Sr Chick,NM,1.0,IND
2114,WMPID4014,"VASQUEZ, GABRIEL",Gabriel Vasquez,NM,2.0,DEM
2116,WMPID2867,"MARTINEZ JOHNSON, ALEXIS",Alexis Martinez Johnson,NM,3.0,REP
3513,WMPID1184,"HERRELL, STELLA YVETTE",Yvette Herrell,NM,2.0,REP


In [1480]:
nm = fuzzy_merge(NM, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [1481]:
nm

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
225,WMPID451,"GARCIA HOLMES, MICHELLE",Michelle Garcia Holmes,NM,1.0,REP,Michelle Garcia Holmes
226,WMPID453,"LEGER FERNANDEZ, TERESA",Teresa Leger,NM,3.0,DEM,Teresa Leger Fernandez
2093,WMPID3359,"STANSBURY, MELANIE",Melanie Stansbury,NM,1.0,DEM,Melanie Ann Stansbury
2109,WMPID4714,"CHICK, CAMERON ALTON MR. SR",Cameron Alton Sr Chick,NM,1.0,IND,
2114,WMPID4014,"VASQUEZ, GABRIEL",Gabriel Vasquez,NM,2.0,DEM,Gabriel Vasquez
2116,WMPID2867,"MARTINEZ JOHNSON, ALEXIS",Alexis Martinez Johnson,NM,3.0,REP,Alexis Martinez Johnson
3513,WMPID1184,"HERRELL, STELLA YVETTE",Yvette Herrell,NM,2.0,REP,Yvette Herrell


In [1482]:
nm = nm.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [1483]:
nm.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID451,"GARCIA HOLMES, MICHELLE",Michelle Garcia Holmes,NM,1.0,REP,Michelle Garcia Holmes,Michelle Garcia Holmes (R) \n\t\t\t\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,Michelle Garcia Holmes,w80182230_headshot.jpg,both
1,WMPID453,"LEGER FERNANDEZ, TERESA",Teresa Leger,NM,3.0,DEM,Teresa Leger Fernandez,Teresa Leger Fernandez (D) \n\t\t\t\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,Teresa Leger Fernandez,Teresa-Leger-Fernandez.PNG,both
2,WMPID3359,"STANSBURY, MELANIE",Melanie Stansbury,NM,1.0,DEM,Melanie Ann Stansbury,Melanie Ann Stansbury (D) \n\t\t\t\t\t\t\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Melanie Ann Stansbury,_Melanie-Stansbury_.jpg,both
3,WMPID4714,"CHICK, CAMERON ALTON MR. SR",Cameron Alton Sr Chick,NM,1.0,IND,,NaN,NaN,NaN,NaN,left_only
4,WMPID4014,"VASQUEZ, GABRIEL",Gabriel Vasquez,NM,2.0,DEM,Gabriel Vasquez,Gabriel Vasquez (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Gabriel Vasquez,Vasquezgabe.jpg,both


In [1484]:
nm._merge.value_counts()

both          6
left_only     1
right_only    0
Name: _merge, dtype: int64

In [1485]:
nm.shape

(7, 12)

In [968]:
NH = h.loc[h['cand_office_st']=='NH']

In [969]:
NH.shape

(14, 6)

In [970]:
NH

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
215,WMPID436,"KUSTER, ANN MCLANE",Ann Kuster,NH,2.0,DEM
1984,WMPID1898,"BURNS, ROBERT ANDREW MR.",Robert Andrew Burns,NH,2.0,REP
1987,WMPID3004,"LEAVITT, KAROLINE",Karoline Leavitt,NH,1.0,REP
1991,WMPID3998,"BRYK, WILLIAM MICHAEL",William Michael Bryk,NH,2.0,DEM
1992,WMPID4345,"POIRIER, DEAN ALLAN MR",Dean Allan Poirier,NH,2.0,REP
1993,WMPID4495,"GREESON, JEFFREY",Jeffrey Greeson,NH,2.0,REP
1994,WMPID3463,"COZZENS, JEFF",Jeff Cozzens,NH,2.0,REP
1995,WMPID4037,"RIDDLE, JASON DANIEL",Jason Daniel Riddle,NH,2.0,REP
1996,WMPID1937,"WILLIAMS, LILY TANG",Lily Tang Williams,NH,2.0,REP
1997,WMPID3349,"BLACK, SCOTT",Scott Black,NH,2.0,REP


In [971]:
nh = fuzzy_merge(NH, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [972]:
nh

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
215,WMPID436,"KUSTER, ANN MCLANE",Ann Kuster,NH,2.0,DEM,Annie Kuster
1984,WMPID1898,"BURNS, ROBERT ANDREW MR.",Robert Andrew Burns,NH,2.0,REP,
1987,WMPID3004,"LEAVITT, KAROLINE",Karoline Leavitt,NH,1.0,REP,Karoline Leavitt
1991,WMPID3998,"BRYK, WILLIAM MICHAEL",William Michael Bryk,NH,2.0,DEM,
1992,WMPID4345,"POIRIER, DEAN ALLAN MR",Dean Allan Poirier,NH,2.0,REP,
1993,WMPID4495,"GREESON, JEFFREY",Jeffrey Greeson,NH,2.0,REP,
1994,WMPID3463,"COZZENS, JEFF",Jeff Cozzens,NH,2.0,REP,
1995,WMPID4037,"RIDDLE, JASON DANIEL",Jason Daniel Riddle,NH,2.0,REP,
1996,WMPID1937,"WILLIAMS, LILY TANG",Lily Tang Williams,NH,2.0,REP,
1997,WMPID3349,"BLACK, SCOTT",Scott Black,NH,2.0,REP,


In [973]:
nh = nh.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [974]:
nh.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID436,"KUSTER, ANN MCLANE",Ann Kuster,NH,2.0,DEM,Annie Kuster,Annie Kuster (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Annie Kuster,AnnieKuster.jpg,both
1,WMPID1898,"BURNS, ROBERT ANDREW MR.",Robert Andrew Burns,NH,2.0,REP,,NaN,NaN,NaN,NaN,left_only
2,WMPID3004,"LEAVITT, KAROLINE",Karoline Leavitt,NH,1.0,REP,Karoline Leavitt,Karoline Leavitt (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Karoline Leavitt,KarolineLeavitt.PNG,both
3,WMPID3998,"BRYK, WILLIAM MICHAEL",William Michael Bryk,NH,2.0,DEM,,NaN,NaN,NaN,NaN,left_only
4,WMPID4345,"POIRIER, DEAN ALLAN MR",Dean Allan Poirier,NH,2.0,REP,,NaN,NaN,NaN,NaN,left_only


In [975]:
nh._merge.value_counts()

left_only     10
both           4
right_only     0
Name: _merge, dtype: int64

In [976]:
nh.shape

(14, 12)

In [943]:
NV = h.loc[h['cand_office_st']=='NV']

In [944]:
NV.shape

(13, 6)

In [945]:
NV

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
227,WMPID2657,"BEST, RUSSELL",Russell Best,NV,2.0,IAP
229,WMPID2734,"PETERS, SAMUEL JAMES MR",Samuel James Peters,NV,4.0,REP
2121,WMPID3760,"BROG, DAVID",David Brog,NV,1.0,REP
2125,WMPID818,"AMODEI, MARK EUGENE",Mark Amodei,NV,2.0,REP
2131,WMPID4785,"KRAUSE, E M",E M Krause,NV,2.0,DEM
2136,WMPID2001,"ROBERTSON, MARK",Mark Robertson,NV,1.0,REP
2137,WMPID3178,"BECKER, APRIL",April Becker,NV,3.0,REP
2144,WMPID3951,"MORRIS, ROBERT BRUCE",Robert Bruce Morris,NV,3.0,NON
2146,WMPID819,"HORSFORD, STEVEN ALEXZANDER",Steven Horsford,NV,4.0,DEM
2147,WMPID3186,"JOHNSON, JOHN",John Johnson,NV,4.0,NPA


In [946]:
nv = fuzzy_merge(NV, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [947]:
nv

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
227,WMPID2657,"BEST, RUSSELL",Russell Best,NV,2.0,IAP,Russell Best
229,WMPID2734,"PETERS, SAMUEL JAMES MR",Samuel James Peters,NV,4.0,REP,
2121,WMPID3760,"BROG, DAVID",David Brog,NV,1.0,REP,
2125,WMPID818,"AMODEI, MARK EUGENE",Mark Amodei,NV,2.0,REP,Mark Amodei
2131,WMPID4785,"KRAUSE, E M",E M Krause,NV,2.0,DEM,
2136,WMPID2001,"ROBERTSON, MARK",Mark Robertson,NV,1.0,REP,Mark Robertson
2137,WMPID3178,"BECKER, APRIL",April Becker,NV,3.0,REP,April Becker
2144,WMPID3951,"MORRIS, ROBERT BRUCE",Robert Bruce Morris,NV,3.0,NON,
2146,WMPID819,"HORSFORD, STEVEN ALEXZANDER",Steven Horsford,NV,4.0,DEM,Steven Horsford
2147,WMPID3186,"JOHNSON, JOHN",John Johnson,NV,4.0,NPA,


In [948]:
nv = nv.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [949]:
nv.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID2657,"BEST, RUSSELL",Russell Best,NV,2.0,IAP,Russell Best,Russell Best (Independent American Party) \n\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Russell Best,Russell_Best.png,both
1,WMPID2734,"PETERS, SAMUEL JAMES MR",Samuel James Peters,NV,4.0,REP,,NaN,NaN,NaN,NaN,left_only
2,WMPID3760,"BROG, DAVID",David Brog,NV,1.0,REP,,NaN,NaN,NaN,NaN,left_only
3,WMPID818,"AMODEI, MARK EUGENE",Mark Amodei,NV,2.0,REP,Mark Amodei,Mark Amodei (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mark Amodei,Mark_armodei.jpg,both
4,WMPID4785,"KRAUSE, E M",E M Krause,NV,2.0,DEM,,NaN,NaN,NaN,NaN,left_only


In [950]:
nv._merge.value_counts()

both          7
left_only     6
right_only    0
Name: _merge, dtype: int64

In [951]:
nv.shape

(13, 12)

In [918]:
NE = h.loc[h['cand_office_st']=='NE']

In [919]:
NE.shape

(8, 6)

In [920]:
NE

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
211,WMPID1900,"ELSE, DAVID J",David Else,NE,3.0,DEM
212,WMPID428,"ELWORTH, MARK",Mark Elworth Jr,NE,3.0,OTH
1971,WMPID1606,"PANSING BROOKS, PATTY",Patty Pansing Brooks,NE,1.0,DEM
1973,WMPID2875,"FLOOD, MIKE",Mike Flood,NE,1.0,REP
1974,WMPID2175,"WEYER, DANIEL FRED",Daniel Fred Weyer,NE,1.0,NaN
1979,WMPID4441,"VARGAS, ANTHONY 'TONY'",Tony Vargas,NE,2.0,DEM
3327,WMPID997,"BACON, DONALD J",Don Bacon,NE,2.0,REP
3328,WMPID998,"SMITH, ADRIAN",Adrian Smith,NE,3.0,REP


In [921]:
ne = fuzzy_merge(NE, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [922]:
ne

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
211,WMPID1900,"ELSE, DAVID J",David Else,NE,3.0,DEM,David Else
212,WMPID428,"ELWORTH, MARK",Mark Elworth Jr,NE,3.0,OTH,Mark Elworth Jr.
1971,WMPID1606,"PANSING BROOKS, PATTY",Patty Pansing Brooks,NE,1.0,DEM,Patty Pansing Brooks
1973,WMPID2875,"FLOOD, MIKE",Mike Flood,NE,1.0,REP,Mike Flood
1974,WMPID2175,"WEYER, DANIEL FRED",Daniel Fred Weyer,NE,1.0,NaN,
1979,WMPID4441,"VARGAS, ANTHONY 'TONY'",Tony Vargas,NE,2.0,DEM,Tony Vargas
3327,WMPID997,"BACON, DONALD J",Don Bacon,NE,2.0,REP,Don Bacon
3328,WMPID998,"SMITH, ADRIAN",Adrian Smith,NE,3.0,REP,Adrian Smith


In [923]:
ne = ne.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [924]:
ne.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID1900,"ELSE, DAVID J",David Else,NE,3.0,DEM,David Else,David Else (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,David Else,David-Else.png,both
1,WMPID428,"ELWORTH, MARK",Mark Elworth Jr,NE,3.0,OTH,Mark Elworth Jr.,Mark Elworth Jr. (Legal Marijuana Now Party) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Mark Elworth Jr.,Mark-Elworth.jpg,both
2,WMPID1606,"PANSING BROOKS, PATTY",Patty Pansing Brooks,NE,1.0,DEM,Patty Pansing Brooks,Patty Pansing Brooks (D) \n\t\t\t\t\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,Patty Pansing Brooks,Patty_Pansing_Brooks.jpg,both
3,WMPID2875,"FLOOD, MIKE",Mike Flood,NE,1.0,REP,Mike Flood,Mike Flood (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mike Flood,Mike-Flood.jpg,both
4,WMPID2175,"WEYER, DANIEL FRED",Daniel Fred Weyer,NE,1.0,NaN,,NaN,NaN,NaN,NaN,left_only


In [925]:
ne._merge.value_counts()

both          7
left_only     1
right_only    0
Name: _merge, dtype: int64

In [926]:
ne.shape

(8, 12)

In [893]:
MT = h.loc[h['cand_office_st']=='MT']

In [894]:
MT.shape

(8, 6)

In [895]:
MT

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
194,WMPID1739,"RANKIN, SAM",Sam Rankin,MT,2.0,LIB
1841,WMPID4833,"LAMB, JOHN",John Lamb,MT,1.0,LIB
1842,WMPID2752,"ZYGMOND, CURT ANDREW MR. SR.",Curt Andrew Sr. Zygmond,MT,2.0,IND
1845,WMPID2524,"TRANEL, MONICA",Monica Tranel,MT,1.0,DEM
1848,WMPID1677,"RONNING, PENNY",Penny Ronning,MT,2.0,DEM
1851,WMPID2996,"BUCHANAN, GARY",Gary Buchanan,MT,2.0,IND
3220,WMPID902,"ROSENDALE, MATT MR.",Matt Rosendale,MT,2.0,REP
3221,WMPID2527,"ZINKE, RYAN K",Ryan Zinke,MT,1.0,UNK


In [896]:
mt = fuzzy_merge(MT, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [897]:
mt

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
194,WMPID1739,"RANKIN, SAM",Sam Rankin,MT,2.0,LIB,Sam Rankin
1841,WMPID4833,"LAMB, JOHN",John Lamb,MT,1.0,LIB,John Lamb
1842,WMPID2752,"ZYGMOND, CURT ANDREW MR. SR.",Curt Andrew Sr. Zygmond,MT,2.0,IND,
1845,WMPID2524,"TRANEL, MONICA",Monica Tranel,MT,1.0,DEM,Monica Tranel
1848,WMPID1677,"RONNING, PENNY",Penny Ronning,MT,2.0,DEM,Penny Ronning
1851,WMPID2996,"BUCHANAN, GARY",Gary Buchanan,MT,2.0,IND,Gary Buchanan
3220,WMPID902,"ROSENDALE, MATT MR.",Matt Rosendale,MT,2.0,REP,Matt Rosendale
3221,WMPID2527,"ZINKE, RYAN K",Ryan Zinke,MT,1.0,UNK,Ryan Zinke


In [898]:
mt = mt.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [899]:
mt.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID1739,"RANKIN, SAM",Sam Rankin,MT,2.0,LIB,Sam Rankin,Sam Rankin (L) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Sam Rankin,Sam_RankinMT.jpeg,both
1,WMPID4833,"LAMB, JOHN",John Lamb,MT,1.0,LIB,John Lamb,John Lamb (L) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,John Lamb,John-Lamb.PNG,both
2,WMPID2752,"ZYGMOND, CURT ANDREW MR. SR.",Curt Andrew Sr. Zygmond,MT,2.0,IND,,NaN,NaN,NaN,NaN,left_only
3,WMPID2524,"TRANEL, MONICA",Monica Tranel,MT,1.0,DEM,Monica Tranel,Monica Tranel (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Monica Tranel,Monica-Tranel.PNG,both
4,WMPID1677,"RONNING, PENNY",Penny Ronning,MT,2.0,DEM,Penny Ronning,Penny Ronning (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Penny Ronning,Penny-Ronning.jpg,both


In [900]:
mt._merge.value_counts()

both          7
left_only     1
right_only    0
Name: _merge, dtype: int64

In [901]:
mt.shape

(8, 12)

In [843]:
MS = h.loc[h['cand_office_st']=='MS']

In [844]:
MS.shape

(12, 6)

In [845]:
MS

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
191,WMPID399,"FLOWERS, BRIAN WESLEY",Brian Flowers,MS,2.0,REP
1810,WMPID4088,"MCCAY, JAMES MARION MR. JR.",James Marion Jr. Mccay,MS,1.0,OTH
1812,WMPID3880,"BLACK, DIANNE DODSON",Dianne Dodson Black,MS,1.0,DEM
1819,WMPID3236,"YOUNG, SHUWASKI",Shuwaski Young,MS,3.0,DEM
1823,WMPID2475,"JOHNSON, ALDEN PATRICK MR.",Alden Patrick Johnson,MS,4.0,LIB
1824,WMPID3543,"HUDSON, JESSE GRAHAM",Jesse Graham Hudson,MS,4.0,IND
1825,WMPID3802,"EZELL, WALTER MICHAEL",Walter Michael Ezell,MS,4.0,REP
1831,WMPID2749,"LEE, AARON PAYTON",Aaron Payton Lee,MS,4.0,IND
1833,WMPID4871,"DUPREE, JOHNNY L",Johnny Dupree,MS,4.0,DEM
3219,WMPID901,"THOMPSON, BENNIE G.",Bennie G Thompson,MS,2.0,DEM


In [846]:
ms = fuzzy_merge(MS, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [847]:
ms

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
191,WMPID399,"FLOWERS, BRIAN WESLEY",Brian Flowers,MS,2.0,REP,Brian Flowers
1810,WMPID4088,"MCCAY, JAMES MARION MR. JR.",James Marion Jr. Mccay,MS,1.0,OTH,
1812,WMPID3880,"BLACK, DIANNE DODSON",Dianne Dodson Black,MS,1.0,DEM,
1819,WMPID3236,"YOUNG, SHUWASKI",Shuwaski Young,MS,3.0,DEM,Shuwaski Young
1823,WMPID2475,"JOHNSON, ALDEN PATRICK MR.",Alden Patrick Johnson,MS,4.0,LIB,
1824,WMPID3543,"HUDSON, JESSE GRAHAM",Jesse Graham Hudson,MS,4.0,IND,
1825,WMPID3802,"EZELL, WALTER MICHAEL",Walter Michael Ezell,MS,4.0,REP,
1831,WMPID2749,"LEE, AARON PAYTON",Aaron Payton Lee,MS,4.0,IND,
1833,WMPID4871,"DUPREE, JOHNNY L",Johnny Dupree,MS,4.0,DEM,Johnny DuPree
3219,WMPID901,"THOMPSON, BENNIE G.",Bennie G Thompson,MS,2.0,DEM,Bennie Thompson


In [848]:
ms = ms.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [849]:
ms.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID399,"FLOWERS, BRIAN WESLEY",Brian Flowers,MS,2.0,REP,Brian Flowers,Brian Flowers (R) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Brian Flowers,BrianFlowers.jpeg,both
1,WMPID4088,"MCCAY, JAMES MARION MR. JR.",James Marion Jr. Mccay,MS,1.0,OTH,,NaN,NaN,NaN,NaN,left_only
2,WMPID3880,"BLACK, DIANNE DODSON",Dianne Dodson Black,MS,1.0,DEM,,NaN,NaN,NaN,NaN,left_only
3,WMPID3236,"YOUNG, SHUWASKI",Shuwaski Young,MS,3.0,DEM,Shuwaski Young,Shuwaski Young (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Shuwaski Young,Shuwaski_Young.JPG,both
4,WMPID2475,"JOHNSON, ALDEN PATRICK MR.",Alden Patrick Johnson,MS,4.0,LIB,,NaN,NaN,NaN,NaN,left_only


In [850]:
ms._merge.value_counts()

left_only     6
both          6
right_only    0
Name: _merge, dtype: int64

In [851]:
ms.shape

(12, 12)

In [804]:
MN = h.loc[h['cand_office_st']=='MN']

In [805]:
MN.shape

(18, 6)

In [820]:
MN

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
179,WMPID363,"KISTNER, TYLER",Tyler Kistner,MN,2.0,REP
180,WMPID367,"MCCOLLUM, BETTY",Betty McCollum,MN,4.0,DEM
181,WMPID375,"FISCHBACH, MICHELLE",Michelle Fischbach,MN,7.0,REP
1705,WMPID2118,"FINSTAD, BRAD",Brad Finstad,MN,1.0,REP
1707,WMPID3308,"ETTINGER, JEFFREY M.",Jeffrey Ettinger,MN,1.0,DFL
1719,WMPID4966,"ABRAHAMSON, BRIAN",Brian Abrahamson,MN,1.0,OTH
1721,WMPID4223,"WEILER, TOM",Tom Weiler,MN,3.0,REP
1727,WMPID2397,"LOR XIONG, MAY",May Lor Xiong,MN,4.0,REP
1730,WMPID4126,"DAVIS, CICELY",Cicely Davis,MN,5.0,REP
1737,WMPID3734,"HENDRICKS, JEANNE",Jeanne Hendricks,MN,6.0,DFL


In [821]:
mn = fuzzy_merge(MN, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [822]:
mn

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
179,WMPID363,"KISTNER, TYLER",Tyler Kistner,MN,2.0,REP,Tyler Kistner
180,WMPID367,"MCCOLLUM, BETTY",Betty McCollum,MN,4.0,DEM,Betty McCollum
181,WMPID375,"FISCHBACH, MICHELLE",Michelle Fischbach,MN,7.0,REP,Michelle Fischbach
1705,WMPID2118,"FINSTAD, BRAD",Brad Finstad,MN,1.0,REP,Brad Finstad
1707,WMPID3308,"ETTINGER, JEFFREY M.",Jeffrey Ettinger,MN,1.0,DFL,Jeff Ettinger
1719,WMPID4966,"ABRAHAMSON, BRIAN",Brian Abrahamson,MN,1.0,OTH,Brian Abrahamson
1721,WMPID4223,"WEILER, TOM",Tom Weiler,MN,3.0,REP,Tom Weiler
1727,WMPID2397,"LOR XIONG, MAY",May Lor Xiong,MN,4.0,REP,May Lor Xiong
1730,WMPID4126,"DAVIS, CICELY",Cicely Davis,MN,5.0,REP,Cicely Davis
1737,WMPID3734,"HENDRICKS, JEANNE",Jeanne Hendricks,MN,6.0,DFL,Jeanne Hendricks


In [823]:
mn = mn.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [824]:
mn.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID363,"KISTNER, TYLER",Tyler Kistner,MN,2.0,REP,Tyler Kistner,Tyler Kistner (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Tyler Kistner,TylerKistner.jpg,both
1,WMPID367,"MCCOLLUM, BETTY",Betty McCollum,MN,4.0,DEM,Betty McCollum,Betty McCollum (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Betty McCollum,2019_Official_Portrait-min.jpg,both
2,WMPID375,"FISCHBACH, MICHELLE",Michelle Fischbach,MN,7.0,REP,Michelle Fischbach,Michelle Fischbach (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Michelle Fischbach,Michelle-Fischbach.PNG,both
3,WMPID2118,"FINSTAD, BRAD",Brad Finstad,MN,1.0,REP,Brad Finstad,Brad Finstad (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Brad Finstad,Brad-Finstad.PNG,both
4,WMPID3308,"ETTINGER, JEFFREY M.",Jeffrey Ettinger,MN,1.0,DFL,Jeff Ettinger,Jeff Ettinger (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Jeff Ettinger,Jeff-Ettinger.PNG,both


In [825]:
mn._merge.value_counts()

both          18
left_only      0
right_only     0
Name: _merge, dtype: int64

In [826]:
mn.shape

(18, 12)

In [793]:
MI = h.loc[h['cand_office_st']=='MI']

In [794]:
MI.shape

(47, 6)

In [795]:
MI

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
166,WMPID334,"HUIZENGA, WILLIAM P",Bill Huizenga,MI,4.0,REP
169,WMPID338,"SCHOLTEN, HILLARY",Hillary Scholten,MI,3.0,DEM
171,WMPID5127,"CANNY, DAVID",David Canny,MI,8.0,LIB
174,WMPID347,"JUNGE, PAUL",Paul Junge,MI,8.0,REP
175,WMPID5154,"SALIBA, MIKE",Mike Saliba,MI,10.0,LIB
176,WMPID351,"MCCLAIN, LISA",Lisa McClain,MI,10.0,REP
1610,WMPID3457,"LORINSER, ROBERT JAMES",Robert James Lorinser,MI,1.0,DEM
1611,WMPID5142,"GALE, ANDREW",Andrew Gale,MI,1.0,LIB
1612,WMPID5124,"HAKOLA, LIZ",Liz Hakola,MI,1.0,OTH
1615,WMPID5128,"HEWER, NATHAN",Nathan Hewer,MI,2.0,LIB


In [796]:
mi = fuzzy_merge(MI, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [797]:
mi

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
166,WMPID334,"HUIZENGA, WILLIAM P",Bill Huizenga,MI,4.0,REP,Bill Huizenga
169,WMPID338,"SCHOLTEN, HILLARY",Hillary Scholten,MI,3.0,DEM,Hillary Scholten
171,WMPID5127,"CANNY, DAVID",David Canny,MI,8.0,LIB,David Canny
174,WMPID347,"JUNGE, PAUL",Paul Junge,MI,8.0,REP,Paul Junge
175,WMPID5154,"SALIBA, MIKE",Mike Saliba,MI,10.0,LIB,Mike Saliba
176,WMPID351,"MCCLAIN, LISA",Lisa McClain,MI,10.0,REP,Lisa McClain
1610,WMPID3457,"LORINSER, ROBERT JAMES",Robert James Lorinser,MI,1.0,DEM,
1611,WMPID5142,"GALE, ANDREW",Andrew Gale,MI,1.0,LIB,Andrew Gale
1612,WMPID5124,"HAKOLA, LIZ",Liz Hakola,MI,1.0,OTH,Liz Hakola
1615,WMPID5128,"HEWER, NATHAN",Nathan Hewer,MI,2.0,LIB,Nathan Hewer


In [798]:
mi = mi.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [799]:
mi.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID334,"HUIZENGA, WILLIAM P",Bill Huizenga,MI,4.0,REP,Bill Huizenga,Bill Huizenga (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Bill Huizenga,Bill_Huizenga.jpg,both
1,WMPID338,"SCHOLTEN, HILLARY",Hillary Scholten,MI,3.0,DEM,Hillary Scholten,Hillary Scholten (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Hillary Scholten,HillaryScholten.jpg,both
2,WMPID5127,"CANNY, DAVID",David Canny,MI,8.0,LIB,David Canny,David Canny (L) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,David Canny,David_Canny.jpeg,both
3,WMPID347,"JUNGE, PAUL",Paul Junge,MI,8.0,REP,Paul Junge,Paul Junge (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Paul Junge,PaulJunge.jpg,both
4,WMPID5154,"SALIBA, MIKE",Mike Saliba,MI,10.0,LIB,Mike Saliba,Mike Saliba (L) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Mike Saliba,Mike_Slaiba_.jpg,both


In [800]:
mi._merge.value_counts()

both          38
left_only      9
right_only     0
Name: _merge, dtype: int64

In [801]:
mi.shape

(47, 12)

In [768]:
MA = h.loc[h['cand_office_st']=='MA']

In [769]:
MA.shape

(28, 6)

In [770]:
MA

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
150,WMPID295,"AUCHINCLOSS, JAKE",Jake Auchincloss,MA,4.0,DEM
151,WMPID301,"HALL, JULIE",Julie Hall,MA,4.0,REP
154,WMPID317,"KEATING, WILLIAM RICHARD",Bill Keating,MA,9.0,DEM
1516,WMPID4290,"MARTILLI, DEAN",Dean Martilli,MA,1.0,REP
1517,WMPID2095,"MARINO, LOUIS G",Louis Marino,MA,2.0,REP
1518,WMPID3437,"SOSSA-PAQUETTE, JEFF",Jeff Sossa-Paquette,MA,2.0,REP
1519,WMPID1913,"BARASSI, NICHOLAS",Nicholas Barassi,MA,2.0,REP
1520,WMPID2476,"TRAN, DEAN",Dean Tran,MA,3.0,REP
1521,WMPID4480,"TOZIER-ROBBINS, MIRANDA L.",Miranda Tozier-Robbins,MA,3.0,DEM
1522,WMPID2104,"BURNS, EMILY",Emily Burns,MA,4.0,REP


In [771]:
ma = fuzzy_merge(MA, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [772]:
ma

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
150,WMPID295,"AUCHINCLOSS, JAKE",Jake Auchincloss,MA,4.0,DEM,Jake Auchincloss
151,WMPID301,"HALL, JULIE",Julie Hall,MA,4.0,REP,
154,WMPID317,"KEATING, WILLIAM RICHARD",Bill Keating,MA,9.0,DEM,Bill Keating
1516,WMPID4290,"MARTILLI, DEAN",Dean Martilli,MA,1.0,REP,Dean James Martilli
1517,WMPID2095,"MARINO, LOUIS G",Louis Marino,MA,2.0,REP,
1518,WMPID3437,"SOSSA-PAQUETTE, JEFF",Jeff Sossa-Paquette,MA,2.0,REP,Jeffrey Sossa-Paquette
1519,WMPID1913,"BARASSI, NICHOLAS",Nicholas Barassi,MA,2.0,REP,
1520,WMPID2476,"TRAN, DEAN",Dean Tran,MA,3.0,REP,Dean Tran
1521,WMPID4480,"TOZIER-ROBBINS, MIRANDA L.",Miranda Tozier-Robbins,MA,3.0,DEM,
1522,WMPID2104,"BURNS, EMILY",Emily Burns,MA,4.0,REP,


In [773]:
ma = ma.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [774]:
ma.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID295,"AUCHINCLOSS, JAKE",Jake Auchincloss,MA,4.0,DEM,Jake Auchincloss,Jake Auchincloss (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Jake Auchincloss,Jake-Auchincloss.jpg,both
1,WMPID301,"HALL, JULIE",Julie Hall,MA,4.0,REP,,NaN,NaN,NaN,NaN,left_only
2,WMPID317,"KEATING, WILLIAM RICHARD",Bill Keating,MA,9.0,DEM,Bill Keating,Bill Keating (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Bill Keating,William_Keating.jpg,both
3,WMPID4290,"MARTILLI, DEAN",Dean Martilli,MA,1.0,REP,Dean James Martilli,Dean James Martilli (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Dean James Martilli,Dean_James_Martilli.jpeg,both
4,WMPID2095,"MARINO, LOUIS G",Louis Marino,MA,2.0,REP,,NaN,NaN,NaN,NaN,left_only


In [775]:
ma._merge.value_counts()

both          16
left_only     12
right_only     0
Name: _merge, dtype: int64

In [776]:
ma.shape

(28, 12)

In [741]:
MD = h.loc[h['cand_office_st']=='MD']

In [742]:
MD.shape

(22, 6)

In [743]:
MD

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
157,WMPID4996,"COLLIER, SCOTT M",Scott M Collier,MD,7.0,REP
159,WMPID320,"PALOMBI, CHRISTOPHER",Chris Palombi,MD,5.0,REP
162,WMPID322,"PARROTT, NEIL CONRAD",Neil Parrott,MD,6.0,REP
163,WMPID326,"COLL, GREGORY THOMAS",Gregory Coll,MD,8.0,REP
1540,WMPID4259,"MIZEUR, HEATHER RENAY",Heather Renay Mizeur,MD,1.0,DEM
1542,WMPID5147,"THIBEAULT, DANIEL FRANK",Daniel Frank Thibeault,MD,1.0,LIB
1543,WMPID805,"RUPPERSBERGER, C. A. DUTCH",Dutch Ruppersberger,MD,2.0,DEM
1544,WMPID3195,"KELLEY, JOE",Joe Kelley,MD,2.0,REP
1548,WMPID3388,"MORGAN, YURIPZY",Yuripzy Morgan,MD,3.0,REP
1550,WMPID1735,"AMBROSE, NICOLEE",Nicolee Ambrose,MD,2.0,REP


In [744]:
md = fuzzy_merge(MD, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [745]:
md

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
157,WMPID4996,"COLLIER, SCOTT M",Scott M Collier,MD,7.0,REP,Scott M. Collier
159,WMPID320,"PALOMBI, CHRISTOPHER",Chris Palombi,MD,5.0,REP,Chris Palombi
162,WMPID322,"PARROTT, NEIL CONRAD",Neil Parrott,MD,6.0,REP,Neil Parrott
163,WMPID326,"COLL, GREGORY THOMAS",Gregory Coll,MD,8.0,REP,Gregory Coll
1540,WMPID4259,"MIZEUR, HEATHER RENAY",Heather Renay Mizeur,MD,1.0,DEM,Heather Mizeur
1542,WMPID5147,"THIBEAULT, DANIEL FRANK",Daniel Frank Thibeault,MD,1.0,LIB,Daniel Thibeault
1543,WMPID805,"RUPPERSBERGER, C. A. DUTCH",Dutch Ruppersberger,MD,2.0,DEM,Dutch Ruppersberger
1544,WMPID3195,"KELLEY, JOE",Joe Kelley,MD,2.0,REP,
1548,WMPID3388,"MORGAN, YURIPZY",Yuripzy Morgan,MD,3.0,REP,Yuripzy Morgan
1550,WMPID1735,"AMBROSE, NICOLEE",Nicolee Ambrose,MD,2.0,REP,Nicolee Ambrose


In [746]:
md = md.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [747]:
md.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID4996,"COLLIER, SCOTT M",Scott M Collier,MD,7.0,REP,Scott M. Collier,Scott M. Collier (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Scott M. Collier,Scott_M._Collier.png,both
1,WMPID320,"PALOMBI, CHRISTOPHER",Chris Palombi,MD,5.0,REP,Chris Palombi,Chris Palombi (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Chris Palombi,palombi.jpg,both
2,WMPID322,"PARROTT, NEIL CONRAD",Neil Parrott,MD,6.0,REP,Neil Parrott,Neil Parrott (R) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Neil Parrott,NeilParrott.jpeg,both
3,WMPID326,"COLL, GREGORY THOMAS",Gregory Coll,MD,8.0,REP,Gregory Coll,Gregory Coll (R) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Gregory Coll,Gregory_Coll2.jpeg,both
4,WMPID4259,"MIZEUR, HEATHER RENAY",Heather Renay Mizeur,MD,1.0,DEM,Heather Mizeur,Heather Mizeur (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Heather Mizeur,HeatherMizeur.jpg,both


In [748]:
md._merge.value_counts()

both          16
left_only      6
right_only     0
Name: _merge, dtype: int64

In [749]:
md.shape

(22, 12)

In [714]:
ME = h.loc[h['cand_office_st']=='ME']

In [715]:
ME.shape

(6, 6)

In [716]:
ME

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
1598,WMPID3545,"THELANDER, EDWIN F MR.",Edwin Thelander,ME,1.0,REP
1600,WMPID3404,"INMAN, HOLLIS CHARLES CHANDLER",Hollis Charles Chandler Inman,ME,1.0,IND
3209,WMPID4254,"POLIQUIN, BRUCE LEE",Bruce Lee Poliquin,ME,2.0,REP
3478,WMPID1146,"PINGREE, CHELLIE M",Chellie Pingree,ME,1.0,DEM
3479,WMPID1147,"GOLDEN, JARED",Jared Golden,ME,2.0,DEM
3480,WMPID1383,"BOND, TIFFANY",Tiffany Bond,ME,2.0,IND


In [717]:
me = fuzzy_merge(ME, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [718]:
me

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
1598,WMPID3545,"THELANDER, EDWIN F MR.",Edwin Thelander,ME,1.0,REP,
1600,WMPID3404,"INMAN, HOLLIS CHARLES CHANDLER",Hollis Charles Chandler Inman,ME,1.0,IND,
3209,WMPID4254,"POLIQUIN, BRUCE LEE",Bruce Lee Poliquin,ME,2.0,REP,Bruce Poliquin
3478,WMPID1146,"PINGREE, CHELLIE M",Chellie Pingree,ME,1.0,DEM,Chellie Pingree
3479,WMPID1147,"GOLDEN, JARED",Jared Golden,ME,2.0,DEM,Jared Golden
3480,WMPID1383,"BOND, TIFFANY",Tiffany Bond,ME,2.0,IND,Tiffany Bond


In [719]:
me = me.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [720]:
me.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID3545,"THELANDER, EDWIN F MR.",Edwin Thelander,ME,1.0,REP,,NaN,NaN,NaN,NaN,left_only
1,WMPID3404,"INMAN, HOLLIS CHARLES CHANDLER",Hollis Charles Chandler Inman,ME,1.0,IND,,NaN,NaN,NaN,NaN,left_only
2,WMPID4254,"POLIQUIN, BRUCE LEE",Bruce Lee Poliquin,ME,2.0,REP,Bruce Poliquin,Bruce Poliquin,https://s3.amazonaws.com/ballotpedia-api4/file...,Bruce Poliquin,Bruce_Poliquin_official_congressional_photo.jpeg,both
3,WMPID1146,"PINGREE, CHELLIE M",Chellie Pingree,ME,1.0,DEM,Chellie Pingree,Chellie Pingree,https://s3.amazonaws.com/ballotpedia-api4/file...,Chellie Pingree,Chellie_Pingree.jpg,both
4,WMPID1147,"GOLDEN, JARED",Jared Golden,ME,2.0,DEM,Jared Golden,Jared Golden,https://s3.amazonaws.com/ballotpedia-api4/file...,Jared Golden,JARED_GOLDEN.jpg,both


In [721]:
me._merge.value_counts()

both          4
left_only     2
right_only    0
Name: _merge, dtype: int64

In [722]:
me.shape

(6, 12)

In [689]:
LA = h.loc[h['cand_office_st']=='LA']

In [690]:
LA.shape

(21, 6)

In [691]:
LA

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
145,WMPID273,"SCALISE, STEVE MR",Steve Scalise,LA,1.0,REP
148,WMPID3163,"BADGER, JOHN ROBERT",John Robert Badger,LA,5.0,IND
1478,WMPID4948,"DARLING, KATIE DARLING",Katie Darling Darling,LA,1.0,DEM
1480,WMPID3568,"CARTER, TROY A. SR.",Troy Sr. Carter,LA,2.0,DEM
1492,WMPID4994,"LUX, DANIEL ANTHONY",Daniel Anthony Lux,LA,2.0,REP
1493,WMPID3837,"GRANGER, DUSTIN",Dustin Granger,LA,3.0,DEM
1494,WMPID3131,"SHAHEEN, JACOB LEWIS",Jacob Lewis Shaheen,LA,3.0,REP
1495,WMPID4934,"LEBRUN, TIA MARIE MRS.",Tia Marie Lebrun,LA,3.0,DEM
1496,WMPID4943,"PAYNE, THOMAS LANE JR",Thomas Lane Jr Payne,LA,3.0,REP
1497,WMPID5068,"HOGGATT, HOLDEN",Holden Hoggatt,LA,3.0,REP


In [692]:
la = fuzzy_merge(LA, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [693]:
la

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
145,WMPID273,"SCALISE, STEVE MR",Steve Scalise,LA,1.0,REP,Steve Scalise
148,WMPID3163,"BADGER, JOHN ROBERT",John Robert Badger,LA,5.0,IND,
1478,WMPID4948,"DARLING, KATIE DARLING",Katie Darling Darling,LA,1.0,DEM,Katie Darling
1480,WMPID3568,"CARTER, TROY A. SR.",Troy Sr. Carter,LA,2.0,DEM,Troy Carter
1492,WMPID4994,"LUX, DANIEL ANTHONY",Daniel Anthony Lux,LA,2.0,REP,
1493,WMPID3837,"GRANGER, DUSTIN",Dustin Granger,LA,3.0,DEM,
1494,WMPID3131,"SHAHEEN, JACOB LEWIS",Jacob Lewis Shaheen,LA,3.0,REP,
1495,WMPID4934,"LEBRUN, TIA MARIE MRS.",Tia Marie Lebrun,LA,3.0,DEM,
1496,WMPID4943,"PAYNE, THOMAS LANE JR",Thomas Lane Jr Payne,LA,3.0,REP,Thomas Payne Jr.
1497,WMPID5068,"HOGGATT, HOLDEN",Holden Hoggatt,LA,3.0,REP,Holden Hoggatt


In [694]:
la = la.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [695]:
la.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID273,"SCALISE, STEVE MR",Steve Scalise,LA,1.0,REP,Steve Scalise,Steve Scalise (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Steve Scalise,Steve_Scalise.jpg,both
1,WMPID3163,"BADGER, JOHN ROBERT",John Robert Badger,LA,5.0,IND,,NaN,NaN,NaN,NaN,left_only
2,WMPID4948,"DARLING, KATIE DARLING",Katie Darling Darling,LA,1.0,DEM,Katie Darling,Katie Darling (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Katie Darling,Katie_Darling1.jpg,both
3,WMPID3568,"CARTER, TROY A. SR.",Troy Sr. Carter,LA,2.0,DEM,Troy Carter,Troy Carter (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Troy Carter,TroyCarterheadshot.jpeg,both
4,WMPID4994,"LUX, DANIEL ANTHONY",Daniel Anthony Lux,LA,2.0,REP,,NaN,NaN,NaN,NaN,left_only


In [696]:
la._merge.value_counts()

both          13
left_only      8
right_only     0
Name: _merge, dtype: int64

In [697]:
la.shape

(21, 12)

In [664]:
KY = h.loc[h['cand_office_st']=='KY']

In [665]:
KY.shape

(14, 6)

In [666]:
KY

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
143,WMPID267,"ROGERS, HAROLD DALLAS",Hal Rogers,KY,5.0,REP
144,WMPID271,"BARR, GARLAND ANDY",Andy Barr,KY,6.0,REP
1442,WMPID2012,"AUSBROOKS, JIMMY CLIFTON",Jimmy Clifton Ausbrooks,KY,1.0,DEM
1451,WMPID3617,"MCGARVEY, MORGAN",Morgan Mcgarvey,KY,3.0,DEM
1452,WMPID4051,"RAY, STUART N.",Stuart Ray,KY,3.0,REP
1456,WMPID803,"MASSIE, THOMAS H.",Thomas Massie,KY,4.0,REP
1459,WMPID3335,"LEHMAN, MATTHEW",Matthew Lehman,KY,4.0,DEM
1462,WMPID4405,"OSBORNE, ETHAN KEITH",Ethan Keith Osborne,KY,4.0,OTH
1466,WMPID2950,"HALBLEIB, CONOR",Conor Halbleib,KY,5.0,DEM
1472,WMPID5112,"CRAVENS, MAURICE RANDALL, II",Maurice Randall Cravens,KY,6.0,W


In [667]:
ky = fuzzy_merge(KY, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [668]:
ky

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
143,WMPID267,"ROGERS, HAROLD DALLAS",Hal Rogers,KY,5.0,REP,Hal Rogers
144,WMPID271,"BARR, GARLAND ANDY",Andy Barr,KY,6.0,REP,Andy Barr
1442,WMPID2012,"AUSBROOKS, JIMMY CLIFTON",Jimmy Clifton Ausbrooks,KY,1.0,DEM,
1451,WMPID3617,"MCGARVEY, MORGAN",Morgan Mcgarvey,KY,3.0,DEM,Morgan McGarvey
1452,WMPID4051,"RAY, STUART N.",Stuart Ray,KY,3.0,REP,Stuart Ray
1456,WMPID803,"MASSIE, THOMAS H.",Thomas Massie,KY,4.0,REP,Thomas Massie
1459,WMPID3335,"LEHMAN, MATTHEW",Matthew Lehman,KY,4.0,DEM,Matthew Lehman
1462,WMPID4405,"OSBORNE, ETHAN KEITH",Ethan Keith Osborne,KY,4.0,OTH,Ethan Osborne
1466,WMPID2950,"HALBLEIB, CONOR",Conor Halbleib,KY,5.0,DEM,Conor Halbleib
1472,WMPID5112,"CRAVENS, MAURICE RANDALL, II",Maurice Randall Cravens,KY,6.0,W,


In [669]:
ky = ky.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [670]:
ky.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID267,"ROGERS, HAROLD DALLAS",Hal Rogers,KY,5.0,REP,Hal Rogers,Hal Rogers (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Hal Rogers,Hal_Rogers.JPG,both
1,WMPID271,"BARR, GARLAND ANDY",Andy Barr,KY,6.0,REP,Andy Barr,Andy Barr (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Andy Barr,Andy_Barr__official_portrait__113th_Congress-7...,both
2,WMPID2012,"AUSBROOKS, JIMMY CLIFTON",Jimmy Clifton Ausbrooks,KY,1.0,DEM,,NaN,NaN,NaN,NaN,left_only
3,WMPID3617,"MCGARVEY, MORGAN",Morgan Mcgarvey,KY,3.0,DEM,Morgan McGarvey,Morgan McGarvey (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Morgan McGarvey,Morgan-McGarvey.jpg,both
4,WMPID4051,"RAY, STUART N.",Stuart Ray,KY,3.0,REP,Stuart Ray,Stuart Ray (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Stuart Ray,Stuart-Ray.PNG,both


In [671]:
ky._merge.value_counts()

both          11
left_only      3
right_only     0
Name: _merge, dtype: int64

In [672]:
ky.shape

(14, 12)

In [636]:
KS = h.loc[h['cand_office_st']=='KS']

In [637]:
KS.shape

(8, 6)

In [638]:
KS

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
137,WMPID254,"MANN, TRACEY ROBERT",Tracey Mann,KS,1.0,REP
139,WMPID256,"LATURNER, JAKE",Jake Laturner,KS,2.0,REP
140,WMPID258,"ADKINS, AMANDA",Amanda Adkins,KS,3.0,REP
1429,WMPID4850,"BEARD, JAMES KENNETH",James Kenneth Beard,KS,1.0,DEM
1430,WMPID3427,"SCHMIDT, PATRICK",Patrick Schmidt,KS,2.0,DEM
1436,WMPID4904,"HERNANDEZ, BOB MR",Bob Hernandez,KS,4.0,DEM
3462,WMPID1128,"DAVIDS, SHARICE",Sharice Davids,KS,3.0,DEM
3463,WMPID1130,"ESTES, RON",Ron Estes,KS,4.0,REP


In [640]:
ks = fuzzy_merge(KS, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [641]:
ks

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
137,WMPID254,"MANN, TRACEY ROBERT",Tracey Mann,KS,1.0,REP,Tracey Mann
139,WMPID256,"LATURNER, JAKE",Jake Laturner,KS,2.0,REP,
140,WMPID258,"ADKINS, AMANDA",Amanda Adkins,KS,3.0,REP,Amanda Adkins
1429,WMPID4850,"BEARD, JAMES KENNETH",James Kenneth Beard,KS,1.0,DEM,
1430,WMPID3427,"SCHMIDT, PATRICK",Patrick Schmidt,KS,2.0,DEM,Patrick Schmidt
1436,WMPID4904,"HERNANDEZ, BOB MR",Bob Hernandez,KS,4.0,DEM,Bob Hernandez
3462,WMPID1128,"DAVIDS, SHARICE",Sharice Davids,KS,3.0,DEM,Sharice Davids
3463,WMPID1130,"ESTES, RON",Ron Estes,KS,4.0,REP,Ron Estes


In [642]:
ks = ks.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [643]:
ks.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID254,"MANN, TRACEY ROBERT",Tracey Mann,KS,1.0,REP,Tracey Mann,Tracey Mann (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Tracey Mann,Tracey-Mann.PNG,both
1,WMPID256,"LATURNER, JAKE",Jake Laturner,KS,2.0,REP,,NaN,NaN,NaN,NaN,left_only
2,WMPID258,"ADKINS, AMANDA",Amanda Adkins,KS,3.0,REP,Amanda Adkins,Amanda Adkins (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Amanda Adkins,80312286_162335658479764_6973397713119346688_n...,both
3,WMPID4850,"BEARD, JAMES KENNETH",James Kenneth Beard,KS,1.0,DEM,,NaN,NaN,NaN,NaN,left_only
4,WMPID3427,"SCHMIDT, PATRICK",Patrick Schmidt,KS,2.0,DEM,Patrick Schmidt,Patrick Schmidt (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Patrick Schmidt,Patrick-Schmidt.PNG,both


In [646]:
ks._merge.value_counts()

both          6
left_only     2
right_only    0
Name: _merge, dtype: int64

In [647]:
ks.shape

(8, 12)

In [611]:
IA = h.loc[h['cand_office_st']=='IA']

In [612]:
IA.shape

(11, 6)

In [613]:
IA

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
116,WMPID209,"ARENHOLZ, ASHLEY HINSON",Ashley Hinson,IA,2.0,REP
117,WMPID212,"FEENSTRA, RANDALL",Randy Feenstra,IA,4.0,REP
1240,WMPID1561,"MATHIS, ELIZABETH",Elizabeth Mathis,IA,2.0,DEM
1242,WMPID3752,"BOHANNAN, CHRISTINA",Christina Bohannan,IA,1.0,DEM
1246,WMPID2331,"NUNN, ZACH",Zach Nunn,IA,3.0,REP
1248,WMPID2613,"O'LEARY, RYAN",Ryan O'Leary,IA,4.0,OTH
1249,WMPID3295,"GROSSMAN, KYLE WILLIAM",Kyle William Grossman,IA,4.0,LIB
1250,WMPID3152,"MELTON, RYAN MICHAEL APPLETON",Ryan Michael Appleton Melton,IA,4.0,DEM
3194,WMPID4985,"HOLDER, BRYAN JACK",Bryan Jack Holder,IA,4.0,OTH
3442,WMPID2556,"MILLER-MEEKS, MARIANNETTE JANE",Mariannette Jane Miller-Meeks,IA,1.0,REP


In [614]:
ia = fuzzy_merge(IA, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [615]:
ia

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
116,WMPID209,"ARENHOLZ, ASHLEY HINSON",Ashley Hinson,IA,2.0,REP,Ashley Hinson
117,WMPID212,"FEENSTRA, RANDALL",Randy Feenstra,IA,4.0,REP,Randy Feenstra
1240,WMPID1561,"MATHIS, ELIZABETH",Elizabeth Mathis,IA,2.0,DEM,
1242,WMPID3752,"BOHANNAN, CHRISTINA",Christina Bohannan,IA,1.0,DEM,Christina Bohannan
1246,WMPID2331,"NUNN, ZACH",Zach Nunn,IA,3.0,REP,Zach Nunn
1248,WMPID2613,"O'LEARY, RYAN",Ryan O'Leary,IA,4.0,OTH,
1249,WMPID3295,"GROSSMAN, KYLE WILLIAM",Kyle William Grossman,IA,4.0,LIB,
1250,WMPID3152,"MELTON, RYAN MICHAEL APPLETON",Ryan Michael Appleton Melton,IA,4.0,DEM,
3194,WMPID4985,"HOLDER, BRYAN JACK",Bryan Jack Holder,IA,4.0,OTH,Bryan Holder
3442,WMPID2556,"MILLER-MEEKS, MARIANNETTE JANE",Mariannette Jane Miller-Meeks,IA,1.0,REP,Mariannette Miller-Meeks


In [616]:
ia = ia.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [617]:
ia.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID209,"ARENHOLZ, ASHLEY HINSON",Ashley Hinson,IA,2.0,REP,Ashley Hinson,Ashley Hinson (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Ashley Hinson,428px-Ashley_Hinson.jpg,both
1,WMPID212,"FEENSTRA, RANDALL",Randy Feenstra,IA,4.0,REP,Randy Feenstra,Randy Feenstra (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Randy Feenstra,Randy_Feenstra.jpg,both
2,WMPID1561,"MATHIS, ELIZABETH",Elizabeth Mathis,IA,2.0,DEM,,NaN,NaN,NaN,NaN,left_only
3,WMPID3752,"BOHANNAN, CHRISTINA",Christina Bohannan,IA,1.0,DEM,Christina Bohannan,Christina Bohannan (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Christina Bohannan,Christina-Bohannon.jpg,both
4,WMPID2331,"NUNN, ZACH",Zach Nunn,IA,3.0,REP,Zach Nunn,Zach Nunn (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Zach Nunn,Zach_Nunn.png,both


In [618]:
ia._merge.value_counts()

both          7
left_only     4
right_only    0
Name: _merge, dtype: int64

In [619]:
ia.shape

(11, 12)

In [585]:
IN = h.loc[h['cand_office_st']=='IN']

In [586]:
IN.shape

(24, 6)

In [587]:
IN

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
131,WMPID237,"MRVAN, FRANK J",Frank J. Mrvan,IN,1.0,DEM
132,WMPID5162,"POWERS, WILLIAM F (BILL)",Bill Powers,IN,1.0,W
133,WMPID244,"SPARTZ, VICTORIA",Victoria Spartz,IN,5.0,REP
135,WMPID248,"BUCSHON, LARRY D.",Larry Bucshon,IN,8.0,REP
1378,WMPID1819,"GREEN, JENNIFER-RUTH",Jennifer-Ruth Green,IN,1.0,REP
1386,WMPID2312,"STEURY, PAUL MR.",Paul Steury,IN,2.0,DEM
1387,WMPID5080,"NISLY, CURT",Curt Nisly,IN,2.0,REP
1390,WMPID4688,"HENRY, WILLIAM E.",Will Henry,IN,2.0,LIB
1393,WMPID3665,"SNYDER, GARY",Gary Snyder,IN,3.0,DEM
1395,WMPID3850,"GOTSCH, NATHAN",Nathan Gotsch,IN,3.0,IND


In [588]:
xin = fuzzy_merge(IN, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [589]:
xin

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
131,WMPID237,"MRVAN, FRANK J",Frank J. Mrvan,IN,1.0,DEM,Frank Mrvan
132,WMPID5162,"POWERS, WILLIAM F (BILL)",Bill Powers,IN,1.0,W,
133,WMPID244,"SPARTZ, VICTORIA",Victoria Spartz,IN,5.0,REP,Victoria Spartz
135,WMPID248,"BUCSHON, LARRY D.",Larry Bucshon,IN,8.0,REP,Larry Bucshon
1378,WMPID1819,"GREEN, JENNIFER-RUTH",Jennifer-Ruth Green,IN,1.0,REP,Jennifer-Ruth Green
1386,WMPID2312,"STEURY, PAUL MR.",Paul Steury,IN,2.0,DEM,Paul Steury
1387,WMPID5080,"NISLY, CURT",Curt Nisly,IN,2.0,REP,
1390,WMPID4688,"HENRY, WILLIAM E.",Will Henry,IN,2.0,LIB,
1393,WMPID3665,"SNYDER, GARY",Gary Snyder,IN,3.0,DEM,Gary Snyder
1395,WMPID3850,"GOTSCH, NATHAN",Nathan Gotsch,IN,3.0,IND,Nathan Gotsch


In [590]:
xin = xin.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [591]:
xin.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID237,"MRVAN, FRANK J",Frank J. Mrvan,IN,1.0,DEM,Frank Mrvan,Frank Mrvan (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Frank Mrvan,Frank__Mrvan.jpg,both
1,WMPID5162,"POWERS, WILLIAM F (BILL)",Bill Powers,IN,1.0,W,,NaN,NaN,NaN,NaN,left_only
2,WMPID244,"SPARTZ, VICTORIA",Victoria Spartz,IN,5.0,REP,Victoria Spartz,Victoria Spartz (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Victoria Spartz,Victoria_Spartz.jpg,both
3,WMPID248,"BUCSHON, LARRY D.",Larry Bucshon,IN,8.0,REP,Larry Bucshon,Larry Bucshon (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Larry Bucshon,Larry_Bucshon.jpg,both
4,WMPID1819,"GREEN, JENNIFER-RUTH",Jennifer-Ruth Green,IN,1.0,REP,Jennifer-Ruth Green,Jennifer-Ruth Green (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Jennifer-Ruth Green,JenniferRuth-Green.PNG,both


In [592]:
xin._merge.value_counts()

both          20
left_only      4
right_only     0
Name: _merge, dtype: int64

In [593]:
xin.shape

(24, 12)

In [560]:
IL = h.loc[h['cand_office_st']=='IL']

In [561]:
IL.shape

(40, 6)

In [562]:
IL

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
123,WMPID222,"QUIGLEY, MIKE",Mike Quigley,IL,5.0,DEM
126,WMPID228,"SEVERINO, JOSEPH",Joseph Severino,IL,10.0,REP
128,WMPID233,"MILLER, MARY",Mary Miller,IL,15.0,REP
129,WMPID236,"KING, ESTHER JOY",Esther Joy King,IL,17.0,REP
1265,WMPID3624,"WRIGHT-MCCULLUM, GLENDA",Glenda Wright-Mccullum,IL,1.0,NON
1271,WMPID2211,"DAVILO, MITCHEL",Mitchel Davilo,IL,1.0,OTH
1278,WMPID2533,"CARLSON, ERIC JOSEPH MR.",Eric Joseph Carlson,IL,1.0,REP
1280,WMPID3132,"JACKSON, JONATHAN",Jonathan Jackson,IL,1.0,DEM
1285,WMPID2508,"REGNIER, JEFFERY",Jeffery Regnier,IL,1.0,REP
1290,WMPID799,"KELLY, ROBIN",Robin Kelly,IL,2.0,DEM


In [563]:
il = fuzzy_merge(IL, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [564]:
il

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
123,WMPID222,"QUIGLEY, MIKE",Mike Quigley,IL,5.0,DEM,Mike Quigley
126,WMPID228,"SEVERINO, JOSEPH",Joseph Severino,IL,10.0,REP,Joseph Severino
128,WMPID233,"MILLER, MARY",Mary Miller,IL,15.0,REP,Mary Miller
129,WMPID236,"KING, ESTHER JOY",Esther Joy King,IL,17.0,REP,Esther Joy King
1265,WMPID3624,"WRIGHT-MCCULLUM, GLENDA",Glenda Wright-Mccullum,IL,1.0,NON,
1271,WMPID2211,"DAVILO, MITCHEL",Mitchel Davilo,IL,1.0,OTH,
1278,WMPID2533,"CARLSON, ERIC JOSEPH MR.",Eric Joseph Carlson,IL,1.0,REP,
1280,WMPID3132,"JACKSON, JONATHAN",Jonathan Jackson,IL,1.0,DEM,Jonathan Jackson
1285,WMPID2508,"REGNIER, JEFFERY",Jeffery Regnier,IL,1.0,REP,
1290,WMPID799,"KELLY, ROBIN",Robin Kelly,IL,2.0,DEM,Robin Kelly


In [565]:
il = il.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [566]:
il.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID222,"QUIGLEY, MIKE",Mike Quigley,IL,5.0,DEM,Mike Quigley,Mike Quigley (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mike Quigley,Mike_Quigley.jpg,both
1,WMPID228,"SEVERINO, JOSEPH",Joseph Severino,IL,10.0,REP,Joseph Severino,Joseph Severino (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Joseph Severino,Joseph-Severino.PNG,both
2,WMPID233,"MILLER, MARY",Mary Miller,IL,15.0,REP,Mary Miller,Mary Miller (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mary Miller,_Mary-Miller_.jpg,both
3,WMPID236,"KING, ESTHER JOY",Esther Joy King,IL,17.0,REP,Esther Joy King,Esther Joy King (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Esther Joy King,Esther-Joy-King.jpg,both
4,WMPID3624,"WRIGHT-MCCULLUM, GLENDA",Glenda Wright-Mccullum,IL,1.0,NON,,NaN,NaN,NaN,NaN,left_only


In [567]:
il._merge.value_counts()

both          31
left_only      9
right_only     0
Name: _merge, dtype: int64

In [568]:
il.shape

(40, 12)

In [531]:
ID = h.loc[h['cand_office_st']=='ID']

In [532]:
ID.shape

(5, 6)

In [533]:
ID

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
1252,WMPID3237,"EVANS, JOE MR.",Joe Evans,ID,1.0,LIB
1254,WMPID4600,"PETERSON, KAYLEE JADE",Kaylee Jade Peterson,ID,1.0,DEM
1258,WMPID1844,"NORMAN, WENDY ANN",Wendy Ann Norman,ID,2.0,DEM
3444,WMPID1105,"FULCHER, RUSSELL",Russ Fulcher,ID,1.0,REP
3445,WMPID1106,"SIMPSON, MICHAEL",Mike Simpson,ID,2.0,REP


In [534]:
xid = fuzzy_merge(ID, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [535]:
xid

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
1252,WMPID3237,"EVANS, JOE MR.",Joe Evans,ID,1.0,LIB,
1254,WMPID4600,"PETERSON, KAYLEE JADE",Kaylee Jade Peterson,ID,1.0,DEM,Kaylee Peterson
1258,WMPID1844,"NORMAN, WENDY ANN",Wendy Ann Norman,ID,2.0,DEM,Wendy Norman
3444,WMPID1105,"FULCHER, RUSSELL",Russ Fulcher,ID,1.0,REP,Russ Fulcher
3445,WMPID1106,"SIMPSON, MICHAEL",Mike Simpson,ID,2.0,REP,


In [537]:
xid = xid.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [538]:
xid.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID3237,"EVANS, JOE MR.",Joe Evans,ID,1.0,LIB,,NaN,NaN,NaN,NaN,left_only
1,WMPID4600,"PETERSON, KAYLEE JADE",Kaylee Jade Peterson,ID,1.0,DEM,Kaylee Peterson,Kaylee Peterson (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Kaylee Peterson,Kaylee_Peterson.PNG,both
2,WMPID1844,"NORMAN, WENDY ANN",Wendy Ann Norman,ID,2.0,DEM,Wendy Norman,Wendy Norman (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Wendy Norman,Wendy-Norman.PNG,both
3,WMPID1105,"FULCHER, RUSSELL",Russ Fulcher,ID,1.0,REP,Russ Fulcher,Russ Fulcher (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Russ Fulcher,RUSS_FULCHER.jpg,both
4,WMPID1106,"SIMPSON, MICHAEL",Mike Simpson,ID,2.0,REP,,NaN,NaN,NaN,NaN,left_only


In [539]:
xid._merge.value_counts()

both          3
left_only     2
right_only    0
Name: _merge, dtype: int64

In [540]:
xid.shape

(5, 12)

In [503]:
HI = h.loc[h['cand_office_st']=='HI']

In [504]:
HI.shape

(5, 6)

In [505]:
HI

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
115,WMPID206,"AKANA, JOSEPH",Joe Akana,HI,2.0,REP
1230,WMPID796,"CASE, EDWARD",Ed Case,HI,1.0,DEM
1231,WMPID2636,"KRESS, CONRAD",Conrad Kress,HI,1.0,REP
1234,WMPID3270,"TOKUDA, JILL NAOMI",Jill Naomi Tokuda,HI,2.0,DEM
3441,WMPID1101,"TIPPENS, MICHELLE ROSE",Michelle Tippens,HI,2.0,LIB


In [506]:
hi = fuzzy_merge(HI, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [507]:
hi

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
115,WMPID206,"AKANA, JOSEPH",Joe Akana,HI,2.0,REP,
1230,WMPID796,"CASE, EDWARD",Ed Case,HI,1.0,DEM,Ed Case
1231,WMPID2636,"KRESS, CONRAD",Conrad Kress,HI,1.0,REP,Conrad Kress
1234,WMPID3270,"TOKUDA, JILL NAOMI",Jill Naomi Tokuda,HI,2.0,DEM,
3441,WMPID1101,"TIPPENS, MICHELLE ROSE",Michelle Tippens,HI,2.0,LIB,Michelle Rose Tippens


In [508]:
hi = hi.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [509]:
hi.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID206,"AKANA, JOSEPH",Joe Akana,HI,2.0,REP,,NaN,NaN,NaN,NaN,left_only
1,WMPID796,"CASE, EDWARD",Ed Case,HI,1.0,DEM,Ed Case,Ed Case (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Ed Case,ED_CASE.jpg,both
2,WMPID2636,"KRESS, CONRAD",Conrad Kress,HI,1.0,REP,Conrad Kress,Conrad Kress (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Conrad Kress,Conrad-Kress.PNG,both
3,WMPID3270,"TOKUDA, JILL NAOMI",Jill Naomi Tokuda,HI,2.0,DEM,,NaN,NaN,NaN,NaN,left_only
4,WMPID1101,"TIPPENS, MICHELLE ROSE",Michelle Tippens,HI,2.0,LIB,Michelle Rose Tippens,Michelle Rose Tippens (L) \n\t\t\t\t\t\t\t\t\...,https://s3.amazonaws.com/ballotpedia-api4/file...,Michelle Rose Tippens,Michelle_Rose_Tippens.jpg,both


In [510]:
hi._merge.value_counts()

both          3
left_only     2
right_only    0
Name: _merge, dtype: int64

In [511]:
hi.shape

(5, 12)

In [478]:
GA = h.loc[h['cand_office_st']=='GA']

In [479]:
GA.shape

(36, 6)

In [480]:
GA

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
100,WMPID180,"ALMONORD, VALBRUN DR.",Valbrun Almonord,GA,3.0,DEM
102,WMPID185,"WILLIAMS, NIKEMA NATASSHA",Nikema Williams,GA,5.0,DEM
103,WMPID189,"GREENE, MARJORIE TAYLOR MRS.",Marjorie Greene,GA,14.0,REP
106,WMPID1902,"GONSALVES, MARK",Mark Gonsalves,GA,7.0,REP
107,WMPID191,"MCCORMICK, RICHARD DEAN DR.",Richard McCormick,GA,6.0,REP
109,WMPID192,"SCOTT, JAMES AUSTIN",Austin Scott,GA,8.0,REP
111,WMPID195,"CLYDE, ANDREW",Andrew Clyde,GA,9.0,REP
112,WMPID197,"JOHNSON, ELIZABETH",Liz Johnson,GA,12.0,DEM
113,WMPID3294,"GONZALES, CAESAR",Caesar Gonzales,GA,13.0,REP
1132,WMPID2811,"HERRING, WADE WILKES MR. II",Wade Wilkes II Herring,GA,1.0,DEM


In [481]:
ga = fuzzy_merge(GA, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [482]:
ga

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
100,WMPID180,"ALMONORD, VALBRUN DR.",Valbrun Almonord,GA,3.0,DEM,
102,WMPID185,"WILLIAMS, NIKEMA NATASSHA",Nikema Williams,GA,5.0,DEM,Nikema Williams
103,WMPID189,"GREENE, MARJORIE TAYLOR MRS.",Marjorie Greene,GA,14.0,REP,Marjorie Taylor Greene
106,WMPID1902,"GONSALVES, MARK",Mark Gonsalves,GA,7.0,REP,Mark Gonsalves
107,WMPID191,"MCCORMICK, RICHARD DEAN DR.",Richard McCormick,GA,6.0,REP,Rich McCormick
109,WMPID192,"SCOTT, JAMES AUSTIN",Austin Scott,GA,8.0,REP,Austin Scott
111,WMPID195,"CLYDE, ANDREW",Andrew Clyde,GA,9.0,REP,Andrew Clyde
112,WMPID197,"JOHNSON, ELIZABETH",Liz Johnson,GA,12.0,DEM,Liz Johnson
113,WMPID3294,"GONZALES, CAESAR",Caesar Gonzales,GA,13.0,REP,Caesar Gonzales
1132,WMPID2811,"HERRING, WADE WILKES MR. II",Wade Wilkes II Herring,GA,1.0,DEM,


In [483]:
ga = ga.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [484]:
ga.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID180,"ALMONORD, VALBRUN DR.",Valbrun Almonord,GA,3.0,DEM,,NaN,NaN,NaN,NaN,left_only
1,WMPID185,"WILLIAMS, NIKEMA NATASSHA",Nikema Williams,GA,5.0,DEM,Nikema Williams,Nikema Williams (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Nikema Williams,WilliamsNikema.jpg,both
2,WMPID189,"GREENE, MARJORIE TAYLOR MRS.",Marjorie Greene,GA,14.0,REP,Marjorie Taylor Greene,Marjorie Taylor Greene (R) \n\t\t\t\t\t\t\t\t...,https://s3.amazonaws.com/ballotpedia-api4/file...,Marjorie Taylor Greene,Marjorie_Taylor_Greene_for_Congress.jpg,both
3,WMPID1902,"GONSALVES, MARK",Mark Gonsalves,GA,7.0,REP,Mark Gonsalves,Mark Gonsalves (R) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Mark Gonsalves,Mark_Gonsalves2.png,both
4,WMPID191,"MCCORMICK, RICHARD DEAN DR.",Richard McCormick,GA,6.0,REP,Rich McCormick,Rich McCormick (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Rich McCormick,rmccormick.PNG,both


In [485]:
ga._merge.value_counts()

both          19
left_only     17
right_only     0
Name: _merge, dtype: int64

In [486]:
ga.shape

(36, 12)

In [453]:
FL = h.loc[h['cand_office_st']=='FL']

In [454]:
FL.shape

(165, 6)

In [455]:
FL

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
66,WMPID4494,"MERK, GREGORY CHARLES CHARLES",Gregory Charles Merk,FL,1.0,REP
67,WMPID122,"LAWSON, ALFRED JR",Al Lawson,FL,5.0,DEM
68,WMPID124,"CAMMACK, KAT",Kat Cammack,FL,3.0,REP
69,WMPID127,"KONIZ, GARY L",Gary Koniz,FL,4.0,W
70,WMPID3921,"AGUILAR, ERICK JAVIER",Erick Javier Aguilar,FL,4.0,REP
...,...,...,...,...,...,...
3429,WMPID1090,"POSEY, BILL",Bill Posey,FL,8.0,REP
3430,WMPID1091,"STEUBE, GREG",Greg Steube,FL,17.0,REP
3431,WMPID2425,"CHERFILUS-MCCORMICK, SHEILA",Sheila Cherfilus-Mccormick,FL,20.0,DEM
3432,WMPID2326,"GRIMES, DEMETRIES ANDREW CDR USN RET",Demetries Andrew Cdr Usn Ret Grimes,FL,15.0,REP


In [456]:
fl = fuzzy_merge(FL, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [457]:
fl

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
66,WMPID4494,"MERK, GREGORY CHARLES CHARLES",Gregory Charles Merk,FL,1.0,REP,
67,WMPID122,"LAWSON, ALFRED JR",Al Lawson,FL,5.0,DEM,
68,WMPID124,"CAMMACK, KAT",Kat Cammack,FL,3.0,REP,Kat Cammack
69,WMPID127,"KONIZ, GARY L",Gary Koniz,FL,4.0,W,Gary Koniz
70,WMPID3921,"AGUILAR, ERICK JAVIER",Erick Javier Aguilar,FL,4.0,REP,
...,...,...,...,...,...,...,...
3429,WMPID1090,"POSEY, BILL",Bill Posey,FL,8.0,REP,Bill Posey
3430,WMPID1091,"STEUBE, GREG",Greg Steube,FL,17.0,REP,Greg Steube
3431,WMPID2425,"CHERFILUS-MCCORMICK, SHEILA",Sheila Cherfilus-Mccormick,FL,20.0,DEM,Sheila Cherfilus-McCormick
3432,WMPID2326,"GRIMES, DEMETRIES ANDREW CDR USN RET",Demetries Andrew Cdr Usn Ret Grimes,FL,15.0,REP,


In [458]:
fl = fl.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [459]:
fl.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID4494,"MERK, GREGORY CHARLES CHARLES",Gregory Charles Merk,FL,1.0,REP,,NaN,NaN,NaN,NaN,left_only
1,WMPID122,"LAWSON, ALFRED JR",Al Lawson,FL,5.0,DEM,,NaN,NaN,NaN,NaN,left_only
2,WMPID124,"CAMMACK, KAT",Kat Cammack,FL,3.0,REP,Kat Cammack,Kat Cammack (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Kat Cammack,KatCammack.jpg,both
3,WMPID127,"KONIZ, GARY L",Gary Koniz,FL,4.0,W,Gary Koniz,Gary Koniz (No Party Affiliation) (Write-in) ...,https://s3.amazonaws.com/ballotpedia-api4/file...,Gary Koniz,GaryKonizFL.png,both
4,WMPID3921,"AGUILAR, ERICK JAVIER",Erick Javier Aguilar,FL,4.0,REP,,NaN,NaN,NaN,NaN,left_only


In [460]:
fl._merge.value_counts()

both          87
left_only     78
right_only     0
Name: _merge, dtype: int64

In [461]:
fl.shape

(165, 12)

In [428]:
CT = h.loc[h['cand_office_st']=='CT']

In [429]:
CT.shape

(12, 6)

In [430]:
CT

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
58,WMPID109,"DELAURO, ROSA L",Rosa L DeLauro,CT,3.0,DEM
59,WMPID111,"PAGLINO, JUSTIN COBB DR MD, PHD",Justin Paglino,CT,3.0,GRE
888,WMPID3980,"LAZOR, LAWRENCE ZIDOVSKY DR",Lawrence Zidovsky Lazor,CT,1.0,REP
889,WMPID786,"COURTNEY, JOSEPH",Joe Courtney,CT,2.0,DEM
890,WMPID4570,"FRANCE, MIKE",Mike France,CT,2.0,REP
892,WMPID4644,"CHAI, AMY FOGELSTROM DR",Amy Fogelstrom Chai,CT,3.0,LIB
893,WMPID4701,"DENARDIS, LESLEY",Lesley Denardis,CT,3.0,REP
895,WMPID1935,"STEVENSON, JAYME MRS.",Jayme Stevenson,CT,4.0,REP
896,WMPID1668,"LOGAN, GEORGE S",George Logan,CT,5.0,REP
3420,WMPID1080,"LARSON, JOHN B",John B Larson,CT,1.0,DEM


In [431]:
ct = fuzzy_merge(CT, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [432]:
ct

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
58,WMPID109,"DELAURO, ROSA L",Rosa L DeLauro,CT,3.0,DEM,Rosa L. DeLauro
59,WMPID111,"PAGLINO, JUSTIN COBB DR MD, PHD",Justin Paglino,CT,3.0,GRE,Justin Paglino
888,WMPID3980,"LAZOR, LAWRENCE ZIDOVSKY DR",Lawrence Zidovsky Lazor,CT,1.0,REP,
889,WMPID786,"COURTNEY, JOSEPH",Joe Courtney,CT,2.0,DEM,Joe Courtney
890,WMPID4570,"FRANCE, MIKE",Mike France,CT,2.0,REP,Mike France
892,WMPID4644,"CHAI, AMY FOGELSTROM DR",Amy Fogelstrom Chai,CT,3.0,LIB,
893,WMPID4701,"DENARDIS, LESLEY",Lesley Denardis,CT,3.0,REP,Lesley DeNardis
895,WMPID1935,"STEVENSON, JAYME MRS.",Jayme Stevenson,CT,4.0,REP,Jayme Stevenson
896,WMPID1668,"LOGAN, GEORGE S",George Logan,CT,5.0,REP,George Logan
3420,WMPID1080,"LARSON, JOHN B",John B Larson,CT,1.0,DEM,John Larson


In [433]:
ct = ct.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [434]:
ct.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID109,"DELAURO, ROSA L",Rosa L DeLauro,CT,3.0,DEM,Rosa L. DeLauro,Rosa L. DeLauro (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Rosa L. DeLauro,Rosa_L._Delauro.jpg,both
1,WMPID111,"PAGLINO, JUSTIN COBB DR MD, PHD",Justin Paglino,CT,3.0,GRE,Justin Paglino,Justin Paglino (G) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Justin Paglino,Justin__Pgaglino___.jpg,both
2,WMPID3980,"LAZOR, LAWRENCE ZIDOVSKY DR",Lawrence Zidovsky Lazor,CT,1.0,REP,,NaN,NaN,NaN,NaN,left_only
3,WMPID786,"COURTNEY, JOSEPH",Joe Courtney,CT,2.0,DEM,Joe Courtney,Joe Courtney (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Joe Courtney,Joe_Courtney.jpg,both
4,WMPID4570,"FRANCE, MIKE",Mike France,CT,2.0,REP,Mike France,Mike France (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mike France,Mike-France.PNG,both


In [435]:
ct._merge.value_counts()

both          10
left_only      2
right_only     0
Name: _merge, dtype: int64

In [436]:
ct.shape

(12, 12)

In [403]:
CO = h.loc[h['cand_office_st']=='CO']

In [404]:
CO.shape

(32, 6)

In [405]:
CO

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
53,WMPID96,"BOEBERT, LAUREN",Lauren Boebert,CO,3.0,REP
55,WMPID98,"MCCORKLE, ISAAC IAN",Isaac McCorkle,CO,4.0,DEM
820,WMPID4752,"QUALTERI, JENNIFER",Jennifer Qualteri,CO,1.0,REP
823,WMPID1868,"MCGONIGAL, RYAN JAMES",Ryan James Mcgonigal,CO,4.0,CON
824,WMPID3792,"YURASH, STEPHEN ANDREW",Stephen Andrew Yurash,CO,2.0,UN
825,WMPID1850,"DAWSON, MARSHALL",Marshall Dawson,CO,2.0,REP
831,WMPID3205,"ZIMMERMAN, MARINA",Marina Zimmerman,CO,3.0,REP
836,WMPID3161,"SKOWRONSKI, KRISTIN ELISE",Kristin Elise Skowronski,CO,3.0,IND
840,WMPID2690,"FRISCH, ADAM",Adam Frisch,CO,3.0,DEM
841,WMPID3843,"VAN RAAM, DOUGLAS S",Douglas Van Raam,CO,3.0,OTH


In [406]:
co = fuzzy_merge(CO, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [407]:
co

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
53,WMPID96,"BOEBERT, LAUREN",Lauren Boebert,CO,3.0,REP,Lauren Boebert
55,WMPID98,"MCCORKLE, ISAAC IAN",Isaac McCorkle,CO,4.0,DEM,Isaac McCorkle
820,WMPID4752,"QUALTERI, JENNIFER",Jennifer Qualteri,CO,1.0,REP,Jennifer Qualteri
823,WMPID1868,"MCGONIGAL, RYAN JAMES",Ryan James Mcgonigal,CO,4.0,CON,Ryan McGonigal
824,WMPID3792,"YURASH, STEPHEN ANDREW",Stephen Andrew Yurash,CO,2.0,UN,
825,WMPID1850,"DAWSON, MARSHALL",Marshall Dawson,CO,2.0,REP,Marshall Dawson
831,WMPID3205,"ZIMMERMAN, MARINA",Marina Zimmerman,CO,3.0,REP,Marina Zimmerman
836,WMPID3161,"SKOWRONSKI, KRISTIN ELISE",Kristin Elise Skowronski,CO,3.0,IND,Kristin Skowronski
840,WMPID2690,"FRISCH, ADAM",Adam Frisch,CO,3.0,DEM,Adam Frisch
841,WMPID3843,"VAN RAAM, DOUGLAS S",Douglas Van Raam,CO,3.0,OTH,


In [408]:
co = co.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [409]:
co.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID96,"BOEBERT, LAUREN",Lauren Boebert,CO,3.0,REP,Lauren Boebert,Lauren Boebert (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Lauren Boebert,LaurenBoebert.jpg,both
1,WMPID98,"MCCORKLE, ISAAC IAN",Isaac McCorkle,CO,4.0,DEM,Isaac McCorkle,Isaac McCorkle (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Isaac McCorkle,372EB166-DD69-41F6-AD08-1A0B0473CBEA.jpeg,both
2,WMPID4752,"QUALTERI, JENNIFER",Jennifer Qualteri,CO,1.0,REP,Jennifer Qualteri,Jennifer Qualteri (R) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Jennifer Qualteri,JenniferQualteri.jpg,both
3,WMPID1868,"MCGONIGAL, RYAN JAMES",Ryan James Mcgonigal,CO,4.0,CON,Ryan McGonigal,Ryan McGonigal (American Constitution Party) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Ryan McGonigal,372EB166-DD69-41F6-AD08-1A0B0473CBEA.jpeg,both
4,WMPID3792,"YURASH, STEPHEN ANDREW",Stephen Andrew Yurash,CO,2.0,UN,,NaN,NaN,NaN,NaN,left_only


In [410]:
co._merge.value_counts()

both          19
left_only     13
right_only     0
Name: _merge, dtype: int64

In [411]:
co.shape

(32, 12)

In [378]:
CA = h.loc[h['cand_office_st']=='CA']

In [379]:
CA.shape

(105, 6)

In [380]:
CA

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
10,WMPID45,"BERA, AMERISH",Ami Bera,CA,6.0,DEM
11,WMPID46,"HAMILTON, TAMIKA",Tamika Hamilton,CA,6.0,REP
16,WMPID52,"OBERNOLTE, JAY",Jay Obernolte,CA,23.0,REP
18,WMPID54,"DESAULNIER, MARK",Mark Desaulnier,CA,10.0,DEM
19,WMPID55,"GARAMENDI, JOHN",John Garamendi,CA,8.0,DEM
...,...,...,...,...,...,...
3406,WMPID1069,"PORTER, KATHERINE",Katie Porter,CA,47.0,DEM
3409,WMPID1070,"BRISCOE, JOHN",John Briscoe,CA,42.0,REP
3411,WMPID1072,"LEVIN, MIKE",Mike Levin,CA,49.0,DEM
3412,WMPID90,"JACOBS, SARA",Sara Jacobs,CA,51.0,DEM


In [381]:
ca = fuzzy_merge(CA, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [382]:
ca

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
10,WMPID45,"BERA, AMERISH",Ami Bera,CA,6.0,DEM,Ami Bera
11,WMPID46,"HAMILTON, TAMIKA",Tamika Hamilton,CA,6.0,REP,Tamika Hamilton
16,WMPID52,"OBERNOLTE, JAY",Jay Obernolte,CA,23.0,REP,Jay Obernolte
18,WMPID54,"DESAULNIER, MARK",Mark Desaulnier,CA,10.0,DEM,Mark DeSaulnier
19,WMPID55,"GARAMENDI, JOHN",John Garamendi,CA,8.0,DEM,John Garamendi
...,...,...,...,...,...,...,...
3406,WMPID1069,"PORTER, KATHERINE",Katie Porter,CA,47.0,DEM,Katie Porter
3409,WMPID1070,"BRISCOE, JOHN",John Briscoe,CA,42.0,REP,John Briscoe
3411,WMPID1072,"LEVIN, MIKE",Mike Levin,CA,49.0,DEM,Mike Levin
3412,WMPID90,"JACOBS, SARA",Sara Jacobs,CA,51.0,DEM,Sara Jacobs


In [383]:
ca = ca.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [384]:
ca.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID45,"BERA, AMERISH",Ami Bera,CA,6.0,DEM,Ami Bera,Ami Bera (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Ami Bera,Ami_Bera.jpg,both
1,WMPID46,"HAMILTON, TAMIKA",Tamika Hamilton,CA,6.0,REP,Tamika Hamilton,Tamika Hamilton (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Tamika Hamilton,TamikaHamiltonCA.jpg,both
2,WMPID52,"OBERNOLTE, JAY",Jay Obernolte,CA,23.0,REP,Jay Obernolte,Jay Obernolte (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Jay Obernolte,Jay-Obernolte.jpg,both
3,WMPID54,"DESAULNIER, MARK",Mark Desaulnier,CA,10.0,DEM,Mark DeSaulnier,Mark DeSaulnier (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mark DeSaulnier,Mark-DeSaulnier.jpg,both
4,WMPID55,"GARAMENDI, JOHN",John Garamendi,CA,8.0,DEM,John Garamendi,John Garamendi (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,John Garamendi,John-Garamendi.PNG,both


In [385]:
ca._merge.value_counts()

both          97
left_only      8
right_only     0
Name: _merge, dtype: int64

In [386]:
ca.shape

(105, 12)

In [353]:
AR = h.loc[h['cand_office_st']=='AR']

In [354]:
AR.shape

(12, 6)

In [355]:
AR

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
3,WMPID28,"CRAWFORD, ERIC ALAN RICK",Rick Crawford,AR,1.0,REP
4,WMPID30,"WOMACK, STEVE",Steve Womack,AR,3.0,REP
458,WMPID3089,"HODGES, MONTE",Monte Hodges,AR,1.0,DEM
460,WMPID2895,"DAUGHERTY, ROGER B",Roger Daugherty,AR,1.0,IND
462,WMPID2822,"HATHAWAY, QUINTESSA DR.",Quintessa Hathaway,AR,2.0,DEM
465,WMPID2319,"WHITE, MICHAEL",Michael White,AR,2.0,LIB
467,WMPID2651,"HAYS, LAUREN",Lauren Hays,AR,3.0,DEM
468,WMPID4244,"WHITE, JOHN",John White,AR,4.0,DEM
469,WMPID3596,"MAXWELL, GREGORY",Gregory Maxwell,AR,4.0,LIB
3170,WMPID863,"HILL, JAMES FRENCH",French Hill,AR,2.0,REP


In [356]:
ar = fuzzy_merge(AR, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [357]:
ar

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
3,WMPID28,"CRAWFORD, ERIC ALAN RICK",Rick Crawford,AR,1.0,REP,Rick Crawford
4,WMPID30,"WOMACK, STEVE",Steve Womack,AR,3.0,REP,Steve Womack
458,WMPID3089,"HODGES, MONTE",Monte Hodges,AR,1.0,DEM,Monte Hodges
460,WMPID2895,"DAUGHERTY, ROGER B",Roger Daugherty,AR,1.0,IND,
462,WMPID2822,"HATHAWAY, QUINTESSA DR.",Quintessa Hathaway,AR,2.0,DEM,Quintessa Hathaway
465,WMPID2319,"WHITE, MICHAEL",Michael White,AR,2.0,LIB,Michael White
467,WMPID2651,"HAYS, LAUREN",Lauren Hays,AR,3.0,DEM,
468,WMPID4244,"WHITE, JOHN",John White,AR,4.0,DEM,John White
469,WMPID3596,"MAXWELL, GREGORY",Gregory Maxwell,AR,4.0,LIB,Gregory Maxwell
3170,WMPID863,"HILL, JAMES FRENCH",French Hill,AR,2.0,REP,French Hill


In [358]:
ar = ar.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [359]:
ar.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID28,"CRAWFORD, ERIC ALAN RICK",Rick Crawford,AR,1.0,REP,Rick Crawford,Rick Crawford (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Rick Crawford,Rick_Crawford_AR.jpg,both
1,WMPID30,"WOMACK, STEVE",Steve Womack,AR,3.0,REP,Steve Womack,Steve Womack (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Steve Womack,Steve_Womack.jpg,both
2,WMPID3089,"HODGES, MONTE",Monte Hodges,AR,1.0,DEM,Monte Hodges,Monte Hodges (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Monte Hodges,Monte-Hodges.jpg,both
3,WMPID2895,"DAUGHERTY, ROGER B",Roger Daugherty,AR,1.0,IND,,NaN,NaN,NaN,NaN,left_only
4,WMPID2822,"HATHAWAY, QUINTESSA DR.",Quintessa Hathaway,AR,2.0,DEM,Quintessa Hathaway,Quintessa Hathaway (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Quintessa Hathaway,Quintessa-Hathaway.jpg,both


In [313]:
AZ = h.loc[h['cand_office_st']=='AZ']

In [314]:
AZ.shape

(23, 6)

In [315]:
AZ

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
5,WMPID35,"GOSAR, PAUL DR.",Paul Gosar,AZ,4.0,REP
7,WMPID2361,"RAMOS, JAVIER",Javier Ramos,AZ,5.0,DEM
470,WMPID3767,"BLACKMAN, WALTER JACK",Walter Jack Blackman,AZ,1.0,REP
472,WMPID1594,"CRANE, ELI",Eli Crane,AZ,2.0,REP
477,WMPID2788,"ENGEL, KIRSTEN",Kirsten Engel,AZ,6.0,DEM
482,WMPID1822,"CISCOMANI, JUAN",Juan Ciscomani,AZ,6.0,REP
490,WMPID1650,"POZZOLO, LUIS B",Luis Pozzolo,AZ,3.0,REP
500,WMPID2750,"LINDBERG, TRAVIS J",Travis Lindberg,AZ,5.0,IND
501,WMPID2663,"SMITH, CLINT WILLIAM",Clint William Smith,AZ,5.0,IND
504,WMPID2790,"HODGE, JEVIN D",Jevin Hodge,AZ,1.0,DEM


In [316]:
az = fuzzy_merge(AZ, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [317]:
az

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
5,WMPID35,"GOSAR, PAUL DR.",Paul Gosar,AZ,4.0,REP,Paul Gosar
7,WMPID2361,"RAMOS, JAVIER",Javier Ramos,AZ,5.0,DEM,
470,WMPID3767,"BLACKMAN, WALTER JACK",Walter Jack Blackman,AZ,1.0,REP,
472,WMPID1594,"CRANE, ELI",Eli Crane,AZ,2.0,REP,Eli Crane
477,WMPID2788,"ENGEL, KIRSTEN",Kirsten Engel,AZ,6.0,DEM,Kirsten Engel
482,WMPID1822,"CISCOMANI, JUAN",Juan Ciscomani,AZ,6.0,REP,Juan Ciscomani
490,WMPID1650,"POZZOLO, LUIS B",Luis Pozzolo,AZ,3.0,REP,Luis Pozzolo
500,WMPID2750,"LINDBERG, TRAVIS J",Travis Lindberg,AZ,5.0,IND,
501,WMPID2663,"SMITH, CLINT WILLIAM",Clint William Smith,AZ,5.0,IND,
504,WMPID2790,"HODGE, JEVIN D",Jevin Hodge,AZ,1.0,DEM,Jevin Hodge


In [318]:
az = az.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [319]:
az.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID35,"GOSAR, PAUL DR.",Paul Gosar,AZ,4.0,REP,Paul Gosar,Paul Gosar (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Paul Gosar,Paul-Gosar.PNG,both
1,WMPID2361,"RAMOS, JAVIER",Javier Ramos,AZ,5.0,DEM,,NaN,NaN,NaN,NaN,left_only
2,WMPID3767,"BLACKMAN, WALTER JACK",Walter Jack Blackman,AZ,1.0,REP,,NaN,NaN,NaN,NaN,left_only
3,WMPID1594,"CRANE, ELI",Eli Crane,AZ,2.0,REP,Eli Crane,Eli Crane (R) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Eli Crane,EliCrane.png,both
4,WMPID2788,"ENGEL, KIRSTEN",Kirsten Engel,AZ,6.0,DEM,Kirsten Engel,Kirsten Engel (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Kirsten Engel,Kirsten-Engel.jpg,both


In [271]:
AL = h.loc[h['cand_office_st']=='AL']

In [272]:
AL.shape

(18, 6)

In [273]:
AL

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation
0,WMPID21,"CARL, JERRY LEE, JR",Jerry Carl,AL,1.0,REP
1,WMPID24,"HARVEY-HALL, PHYLLIS",Phyllis Harvey-Hall,AL,2.0,DEM
2,WMPID27,"SEWELL, TERRI A.",Terri A Sewell,AL,7.0,DEM
433,WMPID4534,"SMITH, JONATHAN",Jonathan Smith,AL,2.0,LIB
434,WMPID769,"ROGERS, MICHAEL",Mike D Rogers,AL,3.0,REP
435,WMPID2759,"BELL, DOUGLAS A.",Douglas Bell,AL,3.0,IND
437,WMPID2793,"VEASEY, KAREN LIN MRS",Karen Lin Veasey,AL,3.0,DEM
441,WMPID4902,"COCHRAN, JOHN CHARLES",John Charles Cochran,AL,4.0,LIB
442,WMPID3188,"STRONG, DALE WHITNEY",Dale Whitney Strong,AL,5.0,REP
448,WMPID2993,"WARNER-STANTON, KATHY L",Kathy Warner-Stanton,AL,5.0,DEM


In [274]:
al = fuzzy_merge(AL, df, 'full_name', 'bp_name', threshold=90)

<ipython-input-47-9ed40aba455b>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
<ipython-input-47-9ed40aba455b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [275]:
al

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches
0,WMPID21,"CARL, JERRY LEE, JR",Jerry Carl,AL,1.0,REP,Jerry Carl
1,WMPID24,"HARVEY-HALL, PHYLLIS",Phyllis Harvey-Hall,AL,2.0,DEM,Phyllis Harvey-Hall
2,WMPID27,"SEWELL, TERRI A.",Terri A Sewell,AL,7.0,DEM,Terri Sewell
433,WMPID4534,"SMITH, JONATHAN",Jonathan Smith,AL,2.0,LIB,
434,WMPID769,"ROGERS, MICHAEL",Mike D Rogers,AL,3.0,REP,Mike Rogers
435,WMPID2759,"BELL, DOUGLAS A.",Douglas Bell,AL,3.0,IND,Douglas Bell
437,WMPID2793,"VEASEY, KAREN LIN MRS",Karen Lin Veasey,AL,3.0,DEM,Lin Veasey
441,WMPID4902,"COCHRAN, JOHN CHARLES",John Charles Cochran,AL,4.0,LIB,
442,WMPID3188,"STRONG, DALE WHITNEY",Dale Whitney Strong,AL,5.0,REP,
448,WMPID2993,"WARNER-STANTON, KATHY L",Kathy Warner-Stanton,AL,5.0,DEM,Kathy Warner-Stanton


In [276]:
al = al.merge(df, left_on='matches', right_on='bp_name', indicator=True, how='left')

In [277]:
al.head()

,wmpid,cand_name,full_name,cand_office_st,cand_office_dist,cand_party_affiliation,matches,bp_name_raw,bp_url,bp_name,bp_url_filename,_merge
0,WMPID21,"CARL, JERRY LEE, JR",Jerry Carl,AL,1.0,REP,Jerry Carl,Jerry Carl (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Jerry Carl,Jerry-Carl.PNG,both
1,WMPID24,"HARVEY-HALL, PHYLLIS",Phyllis Harvey-Hall,AL,2.0,DEM,Phyllis Harvey-Hall,Phyllis Harvey-Hall (D) \n,https://s3.amazonaws.com/ballotpedia-api4/file...,Phyllis Harvey-Hall,phyllis.jpg,both
2,WMPID27,"SEWELL, TERRI A.",Terri A Sewell,AL,7.0,DEM,Terri Sewell,Terri Sewell (D) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Terri Sewell,Terri_Sewell.jpg,both
3,WMPID4534,"SMITH, JONATHAN",Jonathan Smith,AL,2.0,LIB,,NaN,NaN,NaN,NaN,left_only
4,WMPID769,"ROGERS, MICHAEL",Mike D Rogers,AL,3.0,REP,Mike Rogers,Mike Rogers (R) \n\t\t\t\t\t\t\t\t\t\t\t,https://s3.amazonaws.com/ballotpedia-api4/file...,Mike Rogers,Mike-Rogers.jpg,both


### Combine the results

In [ ]:
sen.head()

In [ ]:
house.head()

In [ ]:
congress = pd.concat([sen, house])

In [ ]:
congress.shape

In [ ]:
congress.head()

In [ ]:
congress.to_csv("./bp_congress_scraped_face.csv", index=False)